# **Project Setup**

In [1]:
!pip install -q evaluate rouge-score sacrebleu

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.6 MB/s eta 0:00:00


In [2]:
# !pip uninstall -y unsloth
# !pip install --no-cache-dir "unsloth>=2024.5.0"

In [3]:
%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} {get_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo

In [4]:
import unsloth
import os
import math
import json
import torch
import shutil
import random
import evaluate
import numpy as np
import pandas as pd

from peft import PeftModel
from collections import Counter
from sklearn.cluster import KMeans
from typing import List, Dict, Tuple
from unsloth import FastLanguageModel
from trl import SFTTrainer, SFTConfig
from unsloth import is_bfloat16_supported
from transformers import EarlyStoppingCallback
from safetensors.torch import load_file, save_file
from sentence_transformers import SentenceTransformer
from datasets import load_dataset, DatasetDict, Dataset, load_from_disk

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.benchmark = True             # auto-tuning conv/cuDNN
try:
    torch.set_float32_matmul_precision("high")    # PyTorch >= 2.0
except AttributeError:
    pass

# **Data Loader**

## Indonesian Dataset

In [ ]:
ds_id = load_dataset("indonlp/cendol_collection_v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

data/train-00000-of-00023.parquet:   0%|          | 0.00/114M [00:00<?, ?B/s]

...

data/train-00020-of-00023.parquet:   0%|          | 0.00/252M [00:00<?, ?B/s]

data/train-00021-of-00023.parquet:   0%|          | 0.00/263M [00:00<?, ?B/s]

data/train-00022-of-00023.parquet:   0%|          | 0.00/167M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12810390 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

In [ ]:
print(ds_id)

DatasetDict({
    train: Dataset({
        features: ['dataset_name', 'subset_name', 'prompt_id', 'template_name', 'dataset_key', 'input', 'output'],
        num_rows: 12810390
    })
})


## English Dataset

In [ ]:
ds_en = load_dataset("open-orca/OpenOrca")

README.md: 0.00B [00:00, ?B/s]

1M-GPT4-Augmented.parquet:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

3_5M-GPT3_5-Augmented.parquet:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(ds_en)

DatasetDict({
    train: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 4233923
    })
})


# **Data Preprocess**

## Data Clustering and Sampling

**1. Indonesian Dataset**

In [66]:
dataset_id = load_dataset("lumicero/indonesian_60k")
print(dataset_id)

DatasetDict({
    train: Dataset({
        features: ['dataset_name', 'subset_name', 'prompt_id', 'template_name', 'dataset_key', 'input', 'output'],
        num_rows: 54000
    })
    validation: Dataset({
        features: ['dataset_name', 'subset_name', 'prompt_id', 'template_name', 'dataset_key', 'input', 'output'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['dataset_name', 'subset_name', 'prompt_id', 'template_name', 'dataset_key', 'input', 'output'],
        num_rows: 3000
    })
})


In [67]:
datasets_id = dataset_id.filter(lambda ex: (ex["subset_name"] not in ["indo_puisi", "dolly"] and ex["output"] is not None))

In [68]:
texts = [text for text in datasets_id['train']['output'] if text is not None]
print(texts[0])

Cara untuk membangun ketangguhan emosional begini nih:
1. Mengetahui Arti Ketangguhan dan Memahami Perasaan Anda
Ketahui arti ketangguhan. Kenali perasaan Anda. Terimalah kenyataan bahwa emosi adalah hal yang wajar dalam kehidupan sehari-hari.

2. Mengubah Kebiasaan dari Aspek Fisik
Temukan alasan agar Anda bisa sering tertawa. Tulislah perasaan Anda. Binalah hubungan baik dengan orang lain. Pilihlah makanan yang membuat Anda merasa lebih nyaman. Biasakan berolahraga rutin untuk memperbaiki suasana hati, meningkatkan energi, dan menjaga kesehatan.

3. Mengubah Kebiasaan dari Aspek Mental
Berusahalah meredakan stres. Gunakan umpan balik sebagai kesempatan belajar. Kendalikan hidup Anda. Berpikirlah secara objektif. Berlatihlah meningkatkan kesadaran. Ingatlah bahwa kehidupan ini diciptakan bukan untuk membuat Anda susah. Ucapkan syukur. Maafkan diri sendiri dan orang lain. Didiklah anak Anda agar menjadi pribadi yang tangguh.




In [71]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embeddings_id = model.encode(texts, show_progress_bar=True, batch_size=32)

display(embeddings_id.shape)

Batches:   0%|          | 0/1124 [00:00<?, ?it/s]

(35957, 768)

In [72]:
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels_id = kmeans.fit_predict(embeddings_id)

cluster_distribution = Counter(cluster_labels_id)
print("Cluster size:")
display(cluster_distribution)

Cluster size:


Counter({np.int32(2): 2698,
         np.int32(5): 6610,
         np.int32(0): 6024,
         np.int32(4): 3709,
         np.int32(8): 333,
         np.int32(6): 1739,
         np.int32(9): 3680,
         np.int32(1): 6352,
         np.int32(7): 1607,
         np.int32(3): 3205})

In [73]:
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels_id = kmeans.fit_predict(embeddings_id)

cluster_distribution = Counter(cluster_labels_id)
print("Cluster size:")
display(cluster_distribution)

Cluster size:


Counter({np.int32(2): 2698,
         np.int32(5): 6610,
         np.int32(0): 6024,
         np.int32(4): 3709,
         np.int32(8): 333,
         np.int32(6): 1739,
         np.int32(9): 3680,
         np.int32(1): 6352,
         np.int32(7): 1607,
         np.int32(3): 3205})

In [75]:
import numpy as np

# --- 1. Sampling TEST per cluster (300 per cluster) ---
test_per_cluster = 300
test_indices = []

for cluster_id in range(n_clusters):
    # semua indeks di cluster ini (mengacu ke array `texts` / cluster_labels_id)
    cluster_indices = np.where(cluster_labels_id == cluster_id)[0]

    # ambil max 300, atau sebanyak yang tersedia kalau < 300
    n_test = min(test_per_cluster, len(cluster_indices))

    if n_test > 0:
        test_cluster_indices = np.random.choice(
            cluster_indices,
            size=n_test,
            replace=False
        )
        test_indices.extend(test_cluster_indices)

# unik + bisa diurutkan kalau mau
test_indices = sorted(set(test_indices))

print(f"Total test indices: {len(test_indices)}")

# --- 2. Sampling TRAIN per cluster (seperti kode awal, tapi tidak menyentuh test_indices) ---

total_desired_train = 10000
samples_per_cluster_train = total_desired_train // n_clusters
train_indices = []

for cluster_id in range(n_clusters):
    cluster_indices = np.where(cluster_labels_id == cluster_id)[0]

    # buang yang sudah dipakai sebagai test
    remaining_cluster_indices = np.setdiff1d(
        cluster_indices,
        np.array(test_indices),
        assume_unique=False
    )

    n_train = min(len(remaining_cluster_indices), samples_per_cluster_train)

    if n_train > 0:
        sampled_cluster_indices = np.random.choice(
            remaining_cluster_indices,
            size=n_train,
            replace=False
        )
        train_indices.extend(sampled_cluster_indices)

# Jika masih kurang dari total_desired_train, isi dari cluster besar
if len(train_indices) < total_desired_train:
    remaining = total_desired_train - len(train_indices)
    print(f"Train short by {remaining} samples, attempting to fill from larger clusters...")

    # indeks yang sudah terpakai (train + test), jangan dipakai lagi
    used_indices = set(train_indices) | set(test_indices)
    available_indices = [i for i in range(len(texts)) if i not in used_indices]

    if available_indices:
        additional_samples = np.random.choice(
            available_indices,
            size=min(remaining, len(available_indices)),
            replace=False
        )
        train_indices.extend(additional_samples)

# pastikan tidak ada overlap
train_indices = sorted(set(train_indices))
test_indices = sorted(set(test_indices))

intersection = set(train_indices) & set(test_indices)
assert len(intersection) == 0, "Ada overlap antara train dan test!"

print(f"Final number of train samples: {len(train_indices)}")
print(f"Final number of test samples : {len(test_indices)}")

# --- 3. Map kembali ke indeks asli dataset (karena sebelumnya difilter yang None) ---

original_indices = [
    i for i, text in enumerate(dataset_id['train']['output'])
    if text is not None
]

train_final_indices = [original_indices[i] for i in train_indices]
test_final_indices  = [original_indices[i] for i in test_indices]

# --- 4. Buat dataset train dan test akhir (disjoint) ---

sampled_train_dataset_id = dataset_id['train'].select(train_final_indices)
sampled_test_dataset_id  = dataset_id['train'].select(test_final_indices)

print(f"\nFinal train dataset size: {len(sampled_train_dataset_id)}")
print(f"Final test dataset size : {len(sampled_test_dataset_id)}")

Total test indices: 3000
Train short by 967 samples, attempting to fill from larger clusters...
Final number of train samples: 10000
Final number of test samples : 3000

Final train dataset size: 10000
Final test dataset size : 3000


In [13]:
# # Adjust sampling to get closer to 20k
# total_desired = 10000
# samples_per_cluster = total_desired // n_clusters
# sampled_indices = []

# for cluster_id in range(n_clusters):
#     cluster_indices = np.where(cluster_labels_id == cluster_id)[0]
#     n_samples = min(len(cluster_indices), samples_per_cluster)

#     if n_samples > 0:
#         sampled_cluster_indices = np.random.choice(
#             cluster_indices,
#             size=n_samples,
#             replace=False
#         )
#         sampled_indices.extend(sampled_cluster_indices)

# # If we're still short, sample more from larger clusters
# if len(sampled_indices) < total_desired:
#     remaining = total_desired - len(sampled_indices)
#     print(f"Short by {remaining} samples, attempting to fill from larger clusters...")

#     # Get indices not yet sampled from all clusters
#     used_indices = set(sampled_indices)
#     available_indices = [i for i in range(len(texts)) if i not in used_indices]

#     if available_indices:
#         additional_samples = np.random.choice(
#             available_indices,
#             size=min(remaining, len(available_indices)),
#             replace=False
#         )
#         sampled_indices.extend(additional_samples)

# # Map back to original dataset indices
# original_indices = [i for i, text in enumerate(dataset_id['train']['output']) if text is not None]
# final_indices = [original_indices[i] for i in sampled_indices]

# # Create final sampled dataset
# sampled_dataset_id = dataset_id['train'].select(final_indices)

# print(f"\nFinal number of samples: {len(sampled_dataset_id)}")

**2. English Dataset**

In [53]:
dataset_en = load_dataset("lumicero/english_60k")
print(dataset_en)

DatasetDict({
    train: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 54000
    })
    validation: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 3000
    })
})


In [54]:
datasets_en = dataset_en.filter(lambda ex: ex["response"] is not None)

In [55]:
texts = [text for text in datasets_en['train']['response'] if text is not None]
print(texts[0])

What happens to Michel's body in the movie "Les Diaboliques"?


In [56]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
embeddings_en = model.encode(texts, show_progress_bar=True, batch_size=32)

display(embeddings_en.shape)

Batches:   0%|          | 0/1688 [00:00<?, ?it/s]

(54000, 768)

In [57]:
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels_en = kmeans.fit_predict(embeddings_en)

cluster_distribution = Counter(cluster_labels_en)
print("Cluster size:")
display(cluster_distribution)

Cluster size:


Counter({np.int32(8): 5795,
         np.int32(6): 7202,
         np.int32(5): 6267,
         np.int32(0): 6944,
         np.int32(4): 4040,
         np.int32(7): 6921,
         np.int32(2): 4222,
         np.int32(9): 3296,
         np.int32(1): 3179,
         np.int32(3): 6134})

In [58]:
# Adjust sampling to get closer to 20k
total_desired = 10000
samples_per_cluster = total_desired // n_clusters
sampled_indices = []

for cluster_id in range(n_clusters):
    cluster_indices = np.where(cluster_labels_en == cluster_id)[0]
    n_samples = min(len(cluster_indices), samples_per_cluster)

    if n_samples > 0:
        sampled_cluster_indices = np.random.choice(
            cluster_indices,
            size=n_samples,
            replace=False
        )
        sampled_indices.extend(sampled_cluster_indices)

# If we're still short, sample more from larger clusters
if len(sampled_indices) < total_desired:
    remaining = total_desired - len(sampled_indices)
    print(f"Short by {remaining} samples, attempting to fill from larger clusters...")

    # Get indices not yet sampled from all clusters
    used_indices = set(sampled_indices)
    available_indices = [i for i in range(len(texts)) if i not in used_indices]

    if available_indices:
        additional_samples = np.random.choice(
            available_indices,
            size=min(remaining, len(available_indices)),
            replace=False
        )
        sampled_indices.extend(additional_samples)

# Map back to original dataset indices
original_indices = [i for i, text in enumerate(dataset_en['train']['response']) if text is not None]
final_indices = [original_indices[i] for i in sampled_indices]

# Create final sampled dataset
sampled_dataset_en = dataset_en['train'].select(final_indices)

print(f"\nFinal number of samples: {len(sampled_dataset_en)}")


Final number of samples: 10000


**3. Combine all sampled data**

In [76]:
train_datasets_id = DatasetDict({
    "text": sampled_train_dataset_id
})

data_test_id = DatasetDict({
    "test": sampled_test_dataset_id
})

train_datasets_en = DatasetDict({
    "text": sampled_dataset_en
})

print(train_datasets_id)
print(data_test_id)
print(train_datasets_en)

DatasetDict({
    text: Dataset({
        features: ['dataset_name', 'subset_name', 'prompt_id', 'template_name', 'dataset_key', 'input', 'output'],
        num_rows: 10000
    })
})
DatasetDict({
    test: Dataset({
        features: ['dataset_name', 'subset_name', 'prompt_id', 'template_name', 'dataset_key', 'input', 'output'],
        num_rows: 3000
    })
})
DatasetDict({
    text: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 10000
    })
})


# **Data Formatting**

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-unsloth-bnb-4bit",
    max_seq_length = 2048,    # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,     # A bit more accurate, uses 2x memory
    full_finetuning = False,  # We have full finetuning now!
)

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

**1. Indonesian Dataset**

In [ ]:
print(train_datasets_id)

DatasetDict({
    text: Dataset({
        features: ['dataset_name', 'subset_name', 'prompt_id', 'template_name', 'dataset_key', 'input', 'output'],
        num_rows: 10000
    })
})


In [ ]:
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_id(examples):
    instructions = examples["input"]
    outputs      = examples["output"]
    texts = []

    for input, output in zip(instructions, outputs):
        messages = [
            {"role": "system",    "content": "Kamu adalah asisten yang ahli dalam percakapan bahasa Indonesia."},
            {"role": "user",      "content": input},
            {"role": "assistant", "content": output},
        ]

        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
        )

        texts.append(text + EOS_TOKEN)

    return {"text": texts}

In [ ]:
conversations_id = train_datasets_id.map(formatting_prompts_id, batched=True)

display(conversations_id)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    text: Dataset({
        features: ['dataset_name', 'subset_name', 'prompt_id', 'template_name', 'dataset_key', 'input', 'output', 'text'],
        num_rows: 10000
    })
})

In [ ]:
print(conversations_id["text"]["text"])

Column(['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 01 Dec 2025\n\nKamu adalah asisten yang ahli dalam percakapan bahasa Indonesia.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nCiptakan puisi yang memiliki judul:\nFAF<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIni mungkin puisi yang cocok untuk judul tersebut:\nAndai aku bisa kembali 2 tahun kebelakang saat pertama kali aku bertemu denganmusaat pertama kali aku mengenalmudan saat pertama kali kita memulai kisah indah kitamenuliskan satu per satu hurufmerangkai satu per satu katadan menjabarkannya menjadi satu cerita tanpa batas \n Aku bahagia saat bersamamuaku dapat tertawa lepas tanpa beban saat didekatmudan aku dapat menangis tanpa henti saat kehilanganmuandai aku dapat memutar waktuaku hanya ingin kembali pada massa dimana hanya ada kitaaku ingin kembali pada masa dimana kita dapat hidup berdua tanpa orang lainsaling berbagi apa punberbagi 

In [ ]:
train_id = Dataset.from_dict({
    "text": conversations_id["text"]["text"]
})

print(train_id)

Dataset({
    features: ['text'],
    num_rows: 10000
})


In [ ]:
display(train_id["text"][0])
print(len(train_id["text"]))

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 01 Dec 2025\n\nKamu adalah asisten yang ahli dalam percakapan bahasa Indonesia.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nCiptakan puisi yang memiliki judul:\nFAF<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIni mungkin puisi yang cocok untuk judul tersebut:\nAndai aku bisa kembali 2 tahun kebelakang saat pertama kali aku bertemu denganmusaat pertama kali aku mengenalmudan saat pertama kali kita memulai kisah indah kitamenuliskan satu per satu hurufmerangkai satu per satu katadan menjabarkannya menjadi satu cerita tanpa batas \n Aku bahagia saat bersamamuaku dapat tertawa lepas tanpa beban saat didekatmudan aku dapat menangis tanpa henti saat kehilanganmuandai aku dapat memutar waktuaku hanya ingin kembali pada massa dimana hanya ada kitaaku ingin kembali pada masa dimana kita dapat hidup berdua tanpa orang lainsaling berbagi apa punberbagi rasaberb

10000


**2. English Dataset**

In [ ]:
print(train_datasets_en)

DatasetDict({
    text: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 10000
    })
})


In [ ]:
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_en(examples):
    instructions = examples["question"]
    outputs      = examples["response"]
    texts = []

    for question, response in zip(instructions, outputs):
        messages = [
            {"role": "system",    "content": "You are an assistant who is skilled in English conversation."},
            {"role": "user",      "content": question},
            {"role": "assistant", "content": response},
        ]

        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
        )

        texts.append(text + EOS_TOKEN)

    return {"text": texts}

In [ ]:
conversations_en = train_datasets_en.map(formatting_prompts_en, batched=True)

display(conversations_en)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    text: Dataset({
        features: ['id', 'system_prompt', 'question', 'response', 'text'],
        num_rows: 10000
    })
})

In [ ]:
train_en = Dataset.from_dict({
    "text": conversations_en["text"]["text"]
})

print(train_en)

Dataset({
    features: ['text'],
    num_rows: 10000
})


In [ ]:
display(train_en["text"][0])
print(len(train_en["text"]))

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 01 Dec 2025\n\nYou are an assistant who is skilled in English conversation.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nPlease answer the following question: I\'m taking a test and have to guess the right answer to the question after the article. Article: If you go to England, you may want to visit Big Ben. Big Ben is not the name of a man but a name of the clock in London. It has a long history of about 150 years. It is at the north end of the Palace of Westminster . The big clock has four faces. So you can read the face of the chock everywhere.The hands of it are about four meters long.When the new year comes, it _ to welcome the start of the new year. There are also many fireworks over the clock tower. The big clock makes a very loud sound "Ding dong.Ding dong"-the clock gives the sound every 15 minutes of an hour. Question: What is Big Ben? Options: A: A man B

10000


# **Training**

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,           # Choose any number > 0! Suggested 8, 16, 16, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2025.11.6 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [ ]:
!pip install -q wandb

import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: luminolous (fine-tune-poetry) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
os.environ["WANDB_PROJECT"] = "fp-kk"

In [ ]:
split = train_en.train_test_split(test_size=0.01, seed=3407)

train_dataset = split["train"]
eval_dataset  = split["test"]

print("Train and Eval Size:", len(train_dataset), len(eval_dataset))
print(train_dataset["text"][0])
print(eval_dataset["text"][0])

Train and Eval Size: 9900 100
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 01 Dec 2025

You are an assistant who is skilled in English conversation.<|eot_id|><|start_header_id|>user<|end_header_id|>

Here's a short story: I could hear then boy that say close to me clear as day, however I could not hear the young lady sitting in the back of the room.  WHo am I able to hear louder (A) Boy (B) Lady.  What is the most sensical answer between "Boy" and  "Lady"?
A:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The most sensible answer would be "Boy" since the story mentions that the boy's voice was clear as day, while the young lady's voice in the back of the room could not be heard. Therefore, it can be inferred that the boy's voice was louder and more audible than the lady's voice.<|eot_id|><|eot_id|>
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 01 D

In [ ]:
FastLanguageModel.for_training(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [ ]:
cfg = SFTConfig(
    dataset_text_field          = "text",
    max_seq_length              = 2048,
    packing                     = False,
    dataset_num_proc            = 2,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size  = 2,
    gradient_accumulation_steps = 4,
    warmup_steps                = 10,
    num_train_epochs            = 1.5,
    learning_rate               = 2e-4,
    optim                       = "adamw_8bit",
    weight_decay                = 0.01,
    lr_scheduler_type           = "cosine",
    fp16                        = not is_bfloat16_supported(),
    bf16                        = is_bfloat16_supported(),
    logging_steps               = 1,
    eval_strategy               = "steps",
    eval_steps                  = 10,
    save_strategy               = "no",
    output_dir                  = "./outputs",
    report_to                   = "wandb",
    run_name                    = "lora-en",
    seed                        = 3407,
)

trainer = SFTTrainer(
    model         = model,
    tokenizer     = tokenizer,
    train_dataset = train_dataset,
    eval_dataset  = eval_dataset,
    args          = cfg,
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/9900 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# @markdown Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.161 GB.
2.336 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 9,900 | Num Epochs = 2 | Total steps = 233
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 4 x 1) = 64
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
10,2.167300,2.135862
20,1.906000,2.013244
30,1.878100,1.952291
40,1.960400,1.866516
50,1.753900,1.832550
60,1.746700,1.811090
70,1.638000,1.795538
80,1.761800,1.779335
90,1.781100,1.761490
100,1.781800,1.750430


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


eval/loss,█▆▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,█▂▂▂▂▂▂▂▁▁▁▂▂▂▂▂▂▂▁▁▁▁▁
eval/samples_per_second,▁▇▇▇▇▇▇▇██▇▇▇▇▇▇▇▇█████
eval/steps_per_second,▁▇▇▇▇▇▇▇██▇▇▇▇▇▇▇▇█████
train/epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▇▇▇▇▇█████
train/global_step,▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,█▅▂▂▁▁▁▂▁▃▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▂▅▆████▇▇▇▇▇▇▇▇▆▅▅▅▅▄▄▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train/loss,█▆▅▄▄▄▃▄▄▃▃▃▃▃▂▃▂▃▃▂▂▂▃▂▃▁▂▂▂▁▂▂▁▁▁▁▁▂▂▂
eval/loss,1.70581
eval/runtime,9.4273


In [ ]:
#@markdown Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3356.2383 seconds used for training.
55.94 minutes used for training.
Peak reserved memory = 18.016 GB.
Peak reserved memory for training = 15.68 GB.
Peak reserved memory % of max memory = 81.296 %.
Peak reserved memory for training % of max memory = 70.755 %.


In [ ]:
from pathlib import Path

final_save_dir = "/content/drive/MyDrive/College/KK/Final Project/model/lora-en"
Path(final_save_dir).mkdir(parents=True, exist_ok=True)

model.save_pretrained(final_save_dir)
tokenizer.save_pretrained(final_save_dir)

print("Model has been saved in:", final_save_dir)

Model has been saved in: /content/drive/MyDrive/College/KK/Final Project/model/lora-en


# **Scenario and Evaluation**

In [77]:
# eval_id = load_dataset("lumicero/indonesian_60k")
eval_id = data_test_id
eval_en = load_dataset("lumicero/english_60k")
print(eval_id)
print(eval_en)

DatasetDict({
    test: Dataset({
        features: ['dataset_name', 'subset_name', 'prompt_id', 'template_name', 'dataset_key', 'input', 'output'],
        num_rows: 3000
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 54000
    })
    validation: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 3000
    })
})


In [78]:
eval_id_clean = eval_id.filter(lambda ex: ex["output"] is not None)
eval_en_clean = eval_en.filter(lambda ex: ex["response"] is not None)

Filter:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [80]:
from collections import defaultdict
from datasets import Dataset, DatasetDict

def count_missing_values_any(ds, cols, name="dataset"):
    """
    Cek jumlah None / string kosong di setiap kolom `cols`.
    Bisa untuk Dataset maupun DatasetDict.
    """

    def _count_on_dataset(dset, tag):
        missing_counts = defaultdict(int)
        total = dset.num_rows

        for ex in dset:   # ex = dict per baris
            for col in cols:
                v = ex.get(col, None)
                if v is None:
                    missing_counts[col] += 1
                elif isinstance(v, str) and v.strip() == "":
                    # anggap string kosong sebagai missing value
                    missing_counts[col] += 1

        print(f"\n=== {tag} ===")
        print(f"Total rows: {total}")
        for col in cols:
            n = missing_counts[col]
            print(f"- {col}: {n} missing ({n/total:.4%})")

        return dict(missing_counts)

    # Kalau DatasetDict: loop per split
    if isinstance(ds, DatasetDict):
        results = {}
        for split_name, dset in ds.items():
            tag = f"{name}[{split_name}]"
            results[split_name] = _count_on_dataset(dset, tag)
        return results

    # Kalau Dataset biasa
    elif isinstance(ds, Dataset):
        return _count_on_dataset(ds, name)

    else:
        raise TypeError(f"Object type {type(ds)} tidak didukung. Harus Dataset atau DatasetDict.")

# Dataset Indonesia (eval_id_clean) – kolom: input, output
cols_id = ["input", "output"]
missing_id = count_missing_values_any(eval_id_clean, cols_id, name="eval_id_clean")

# Dataset Inggris (eval_en_clean) – kolom: question, response
cols_en = ["question", "response"]
missing_en = count_missing_values_any(eval_en_clean, cols_en, name="eval_en_clean")


=== eval_id_clean[test] ===
Total rows: 3000
- input: 0 missing (0.0000%)
- output: 0 missing (0.0000%)

=== eval_en_clean[train] ===
Total rows: 54000
- question: 0 missing (0.0000%)
- response: 0 missing (0.0000%)

=== eval_en_clean[validation] ===
Total rows: 3000
- question: 0 missing (0.0000%)
- response: 0 missing (0.0000%)

=== eval_en_clean[test] ===
Total rows: 3000
- question: 0 missing (0.0000%)
- response: 0 missing (0.0000%)


In [81]:
for i in range(3000):
    print("=== SAMPLE", i, "===")
    print("Q_ID:", eval_id_clean["test"][i]["input"])
    print("GT  :", eval_id_clean["test"][i]["output"])
    # print("PRED:", results_detail["predictions"][i])
    print()

Output streaming akan dipotong hingga 5000 baris terakhir.

Sampai saat ini,Negeri besar dimekarkan menjadi Kecamatan dengan wilayah administratif membawahi 9 Kampung yang berada di Negeri Besar yaitu:

=== SAMPLE 1934 ===
Q_ID: Coba ceritain soal Katedral Tallinn
GT  : Katedral Santo Petrus dan Paulus adalah sebuah gereja katedral Katolik yang terletak di Tallinn, Estonia.

Agama Katolik disebarkan ke Estonia oleh Tentara Salib Utara dan mendominasi kehidupan keagamaan pada abad pertengahan. Namun, setelah pergerakan Reformasi pada abad ke-16, agama Lutheranisme menjadi agama mayoritas, dan pada masa kekuasaan Swedia, agama Katolik dilarang.

Setelah Estonia direbut oleh Kekaisaran Rusia selama Perang Utara Bebas, kebebasan beragama kembali ditegakkan. Pada tahun 1799, paroki Katolik telah menjadi cukup besar sehingga mereka memperoleh gedung bekas ruang makan Biara Santa Katarian yang sudah ditutup. Gedung ini lalu menjadi tempat ibadah baru. Pada tahun 1841, gereja ini sudah terlalu

## Test Model

In [ ]:
from sacrebleu.metrics import BLEU
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import random
import transformers
import os
from unsloth import FastLanguageModel
from rouge_score import rouge_scorer   # <-- NEW

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    transformers.set_seed(seed)

# System prompt per bahasa
SYSTEM_PROMPT_ID = (
    "Anda adalah asisten AI yang membantu pengguna dalam bahasa Indonesia. "
)

SYSTEM_PROMPT_EN = (
    "You are a helpful AI assistant that communicates in English. "
)

MAX_SEQ_LEN      = 1024   # sama dengan di FastLanguageModel.from_pretrained
MAX_NEW_TOKENS   = 128
MAX_INPUT_LENGTH = MAX_SEQ_LEN - MAX_NEW_TOKENS  # supaya total <= 1024

def make_collate_fn(tokenizer, system_prompt, schema):
    def collate_fn(batch):
        input_id_list = []
        references = []

        for item in batch:
            if schema == "id":
                input_text = item["input"]      # ID: input/output
                reference  = item["output"]
            elif schema == "en":
                input_text = item["question"]   # EN: question/response
                reference  = item["response"]
            else:
                raise ValueError(f"Unknown schema: {schema}")

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": input_text},
            ]

            # 1) Tokenisasi per-sample TANPA padding batch
            ids = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
                truncation=True,
                max_length=MAX_INPUT_LENGTH,
            )[0]  # shape: (seq_len,)

            input_id_list.append(ids)
            references.append(reference)

        # 2) Hitung panjang maksimum di batch
        max_len = max(t.shape[0] for t in input_id_list)
        pad_id  = tokenizer.pad_token_id
        batch_size = len(input_id_list)

        # 3) Siapkan tensor batched dengan LEFT PADDING manual
        input_ids = torch.full(
            (batch_size, max_len),
            pad_id,
            dtype=torch.long,
        )
        attention_mask = torch.zeros(
            (batch_size, max_len),
            dtype=torch.long,
        )

        for i, ids in enumerate(input_id_list):
            seq_len = ids.shape[0]
            # left padding: isi dari kanan
            input_ids[i, -seq_len:] = ids
            attention_mask[i, -seq_len:] = 1

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "references": references,
        }

    return collate_fn

def evaluate_model(model, tokenizer, eval_dataset, system_prompt,
                   batch_size=64, max_allowed_length=24, schema="en"):
    model.eval()
    all_predictions = []
    all_references = []        # untuk BLEU (nested list)
    all_references_text = []   # untuk ROUGE (list string)

    collate_fn = make_collate_fn(tokenizer, system_prompt, schema)

    dataloader = DataLoader(
        eval_dataset,
        num_workers=8,              # 0 dulu biar error kelihatan jelas
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=False,
        pin_memory=True,
    )

    bleu = BLEU()
    pbar = tqdm(dataloader)

    for batch in pbar:
        input_ids = batch["input_ids"].to(DEVICE)          # shape: (B, L)
        attention_mask = batch["attention_mask"].to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_allowed_length,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=False,
                use_cache=True,
            )

        # semua sequence punya panjang awal yang sama: L
        seq_len = input_ids.shape[1]

        predictions = []
        for i, output in enumerate(outputs):
            # ambil token hasil generate saja (setelah input penuh)
            generated_tokens = output[seq_len:]
            decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True)
            predictions.append(decoded)

        all_predictions.extend(predictions)
        all_references.extend([[ref] for ref in batch["references"]])
        all_references_text.extend(batch["references"])

    # ====== BLEU ======
    final_bleu = bleu.corpus_score(all_predictions, all_references)

    # ====== ROUGE-1, ROUGE-2, ROUGE-L ======
    scorer = rouge_scorer.RougeScorer(
        ["rouge1", "rouge2", "rougeL"],
        use_stemmer=True,
    )

    rouge1_list, rouge2_list, rougeL_list = [], [], []

    for pred, ref in zip(all_predictions, all_references_text):
        scores = scorer.score(ref, pred)
        rouge1_list.append(scores["rouge1"].fmeasure)
        rouge2_list.append(scores["rouge2"].fmeasure)
        rougeL_list.append(scores["rougeL"].fmeasure)

    # kali 100 supaya bentuknya persen seperti BLEU
    rouge1 = float(np.mean(rouge1_list) * 100)
    rouge2 = float(np.mean(rouge2_list) * 100)
    rougeL = float(np.mean(rougeL_list) * 100)

    return {
        "bleu": final_bleu.score,
        "rouge1": rouge1,
        "rouge2": rouge2,
        "rougeL": rougeL,
        "predictions": all_predictions,
        "references": all_references,
    }


def evaluate_model_multiple_runs(
    model,
    tokenizer,
    eval_dataset,
    system_prompt,
    n_runs=3,
    batch_size=128,
    max_allowed_length=24,
    schema="en",
):
    all_results = []

    for run in range(n_runs):
        set_seed(42 + run)
        results = evaluate_model(
            model,
            tokenizer,
            eval_dataset,
            system_prompt=system_prompt,
            batch_size=batch_size,
            max_allowed_length=max_allowed_length,
            schema=schema,
        )
        all_results.append(results)

        bleu_scores   = [r["bleu"]   for r in all_results]
        rouge1_scores = [r["rouge1"] for r in all_results]
        rouge2_scores = [r["rouge2"] for r in all_results]
        rougeL_scores = [r["rougeL"] for r in all_results]

        print(
            f"Run {run + 1}/{n_runs} - "
            f"BLEU: {np.mean(bleu_scores):.4f} ± {np.std(bleu_scores):.4f} | "
            f"R1: {np.mean(rouge1_scores):.4f} | "
            f"R2: {np.mean(rouge2_scores):.4f} | "
            f"RL: {np.mean(rougeL_scores):.4f}"
        )

    bleu_scores   = [r["bleu"]   for r in all_results]
    rouge1_scores = [r["rouge1"] for r in all_results]
    rouge2_scores = [r["rouge2"] for r in all_results]
    rougeL_scores = [r["rougeL"] for r in all_results]

    return {
        "bleu_mean":   float(np.mean(bleu_scores)),
        "bleu_std":    float(np.std(bleu_scores)),
        "rouge1_mean": float(np.mean(rouge1_scores)),
        "rouge1_std":  float(np.std(rouge1_scores)),
        "rouge2_mean": float(np.mean(rouge2_scores)),
        "rouge2_std":  float(np.std(rouge2_scores)),
        "rougeL_mean": float(np.mean(rougeL_scores)),
        "rougeL_std":  float(np.std(rougeL_scores)),
    }

MODEL_DIR  = "unsloth/Qwen2.5-3B-Instruct"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_DIR,
    max_seq_length=MAX_SEQ_LEN,
    dtype=None,
    load_in_4bit=True,
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.padding_side = "left"

model.eval()
FastLanguageModel.for_inference(model)

results_en_detail = evaluate_model(
    model,
    tokenizer,
    eval_en_clean["test"],
    system_prompt=SYSTEM_PROMPT_EN,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="en",
)

results_id_detail = evaluate_model(
    model,
    tokenizer,
    eval_id_clean["test"],
    system_prompt=SYSTEM_PROMPT_ID,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="id",
)

print("\nFinal Results (test split):")
print(
    f"English (eval_en_clean['test']): "
    f"BLEU = {results_en_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_en_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_en_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_en_detail['rougeL']:.4f}"
)
print(
    f"Indonesian (eval_id_clean['test']): "
    f"BLEU = {results_id_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_id_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_id_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_id_detail['rougeL']:.4f}"
)

==((====))==  Unsloth 2025.11.6: Fast Qwen2 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 34/34 [04:53<00:00,  8.64s/it]



Final Results (test split):
English (eval_en_clean['test']): BLEU = 40.6667, ROUGE-1 = 38.9121, ROUGE-2 = 21.8046, ROUGE-L = 30.2506
Indonesian (eval_id_clean['test']): BLEU = 15.6511, ROUGE-1 = 14.2183, ROUGE-2 = 2.6392, ROUGE-L = 10.1113


In [ ]:
for i in range(3):
    print("=== SAMPLE", i, "===")
    print("Q_ID:", eval_en_clean["test"][i]["question"])
    print("GT  :", eval_en_clean["test"][i]["response"])
    print("PRED:", results_en_detail["predictions"][i])
    print()

=== SAMPLE 0 ===
Q_ID: In these two sentences (1) Tomorrow the doctor will take the cast off her arm. (2) Mother took our plates away and came back with some fruit for us to eat., does the word take mean the same thing?
GT  : No, the word "take" means different things in these two sentences. In the first sentence, "take off" means to remove the cast from the arm. In the second sentence, "took away" means to remove the plates from the table, and "came back with" means to bring something back to the table. So, even though both sentences use the word "take," they have different meanings depending on the context.
PRED: No, in the two sentences provided, the word "take" is not used with the same meaning.

In the first sentence: "Tomorrow the doctor will take the cast off her arm."
The word "take" here means to remove or remove something from a place or position. It's used in the context of medical treatment, indicating that the cast will be removed by the doctor.

In the second sentence: "M

In [ ]:
for i in range(3):
    print("=== SAMPLE", i, "===")
    print("Q_ID:", eval_id_clean["test"][i]["input"])
    print("GT  :", eval_id_clean["test"][i]["output"])
    print("PRED:", results_id_detail["predictions"][i])
    print()

=== SAMPLE 0 ===
Q_ID: Yuk, bikin puisi yang bermakna!
GT  : Nih, ada puisi yang bisa bermakna: Sudah lelah kucari tambatan hati 
 Memaksaku agar tak terhenti 
 Terus menerus dan terus berlari 
 Menuntut akan sebuah imaji 
 Aku sadar aku tak bisa 
 Aku mengerti akan sulit terasa 
 Tapi dirimu anggapku bisa 
 Meraih semua dalam angan di kepala 
 Aku akui tak ada sepertimu 
 Ringankan beban dan uluran tanganmu 
 Untuk aku yang lemah tanpamu 
 Untuk aku yang slalu membebanimu 
 Berlalu sudah hari demi hari 
 Begitu setianya kau menemani 
 Begitu eratnya bagaikan tali 
 Kau teman sejati, kau kawan abadi
PRED: Tentu, saya akan mencoba. Mari kita buat puisi tentang keindahan alam:

Di tepi sungai yang tenang,
Bunga mawar berdiri rapi.
Sungai mengalir dengan lembut,
Menghembuskan arwah kejernihan.

Hembusan angin yang sejuk,
Menyapa pohon-pohon yang rindu.
Berdiam di sana, menikmati,
Keindahan alam yang luar biasa.

Pondok bambu yang indah,
Dipenuhi oleh burung

=== SAMPLE 1 ===
Q_ID: Yuk, bi

## Base Model

In [40]:
from sacrebleu.metrics import BLEU
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import random
import transformers
import os
from unsloth import FastLanguageModel
from rouge_score import rouge_scorer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    transformers.set_seed(seed)

# System prompt per bahasa
SYSTEM_PROMPT_ID = (
    "Anda adalah asisten AI yang membantu pengguna dalam bahasa Indonesia. "
)

SYSTEM_PROMPT_EN = (
    "You are a helpful AI assistant that communicates in English. "
)

MAX_SEQ_LEN      = 1024   # sama dengan di FastLanguageModel.from_pretrained
MAX_NEW_TOKENS   = 128
MAX_INPUT_LENGTH = MAX_SEQ_LEN - MAX_NEW_TOKENS  # supaya total <= 1024

def make_collate_fn(tokenizer, system_prompt, schema):
    def collate_fn(batch):
        input_id_list = []
        references = []

        for item in batch:
            if schema == "id":
                input_text = item["input"]      # ID: input/output
                reference  = item["output"]
            elif schema == "en":
                input_text = item["question"]   # EN: question/response
                reference  = item["response"]
            else:
                raise ValueError(f"Unknown schema: {schema}")

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": input_text},
            ]

            # 1) Tokenisasi per-sample TANPA padding batch
            ids = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
                truncation=True,
                max_length=MAX_INPUT_LENGTH,
            )[0]  # shape: (seq_len,)

            input_id_list.append(ids)
            references.append(reference)

        # 2) Hitung panjang maksimum di batch
        max_len = max(t.shape[0] for t in input_id_list)
        pad_id  = tokenizer.pad_token_id
        batch_size = len(input_id_list)

        # 3) Siapkan tensor batched dengan LEFT PADDING manual
        input_ids = torch.full(
            (batch_size, max_len),
            pad_id,
            dtype=torch.long,
        )
        attention_mask = torch.zeros(
            (batch_size, max_len),
            dtype=torch.long,
        )

        for i, ids in enumerate(input_id_list):
            seq_len = ids.shape[0]
            # left padding: isi dari kanan
            input_ids[i, -seq_len:] = ids
            attention_mask[i, -seq_len:] = 1

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "references": references,
        }

    return collate_fn

def evaluate_model(model, tokenizer, eval_dataset, system_prompt,
                   batch_size=64, max_allowed_length=24, schema="en"):
    model.eval()
    all_predictions = []
    all_references = []        # untuk BLEU (nested list)
    all_references_text = []   # untuk ROUGE (list string)

    collate_fn = make_collate_fn(tokenizer, system_prompt, schema)

    dataloader = DataLoader(
        eval_dataset,
        num_workers=8,              # 0 dulu biar error kelihatan jelas
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=False,
        pin_memory=True,
    )

    bleu = BLEU()
    pbar = tqdm(dataloader)

    for batch in pbar:
        input_ids = batch["input_ids"].to(DEVICE)          # shape: (B, L)
        attention_mask = batch["attention_mask"].to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_allowed_length,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=False,
                use_cache=True,
            )

        # semua sequence punya panjang awal yang sama: L
        seq_len = input_ids.shape[1]

        predictions = []
        for i, output in enumerate(outputs):
            # ambil token hasil generate saja (setelah input penuh)
            generated_tokens = output[seq_len:]
            decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True)
            predictions.append(decoded)

        all_predictions.extend(predictions)
        all_references.extend([[ref] for ref in batch["references"]])
        all_references_text.extend(batch["references"])

    # ====== BLEU ======
    final_bleu = bleu.corpus_score(all_predictions, all_references)

    # ====== ROUGE-1, ROUGE-2, ROUGE-L ======
    scorer = rouge_scorer.RougeScorer(
        ["rouge1", "rouge2", "rougeL"],
        use_stemmer=True,
    )

    rouge1_list, rouge2_list, rougeL_list = [], [], []

    for pred, ref in zip(all_predictions, all_references_text):
        scores = scorer.score(ref, pred)
        rouge1_list.append(scores["rouge1"].fmeasure)
        rouge2_list.append(scores["rouge2"].fmeasure)
        rougeL_list.append(scores["rougeL"].fmeasure)

    # kali 100 supaya bentuknya persen seperti BLEU
    rouge1 = float(np.mean(rouge1_list) * 100)
    rouge2 = float(np.mean(rouge2_list) * 100)
    rougeL = float(np.mean(rougeL_list) * 100)

    return {
        "bleu": final_bleu.score,
        "rouge1": rouge1,
        "rouge2": rouge2,
        "rougeL": rougeL,
        "predictions": all_predictions,
        "references": all_references,
    }


def evaluate_model_multiple_runs(
    model,
    tokenizer,
    eval_dataset,
    system_prompt,
    n_runs=3,
    batch_size=128,
    max_allowed_length=24,
    schema="en",
):
    all_results = []

    for run in range(n_runs):
        set_seed(42 + run)
        results = evaluate_model(
            model,
            tokenizer,
            eval_dataset,
            system_prompt=system_prompt,
            batch_size=batch_size,
            max_allowed_length=max_allowed_length,
            schema=schema,
        )
        all_results.append(results)

        bleu_scores   = [r["bleu"]   for r in all_results]
        rouge1_scores = [r["rouge1"] for r in all_results]
        rouge2_scores = [r["rouge2"] for r in all_results]
        rougeL_scores = [r["rougeL"] for r in all_results]

        print(
            f"Run {run + 1}/{n_runs} - "
            f"BLEU: {np.mean(bleu_scores):.4f} ± {np.std(bleu_scores):.4f} | "
            f"R1: {np.mean(rouge1_scores):.4f} | "
            f"R2: {np.mean(rouge2_scores):.4f} | "
            f"RL: {np.mean(rougeL_scores):.4f}"
        )

    bleu_scores   = [r["bleu"]   for r in all_results]
    rouge1_scores = [r["rouge1"] for r in all_results]
    rouge2_scores = [r["rouge2"] for r in all_results]
    rougeL_scores = [r["rougeL"] for r in all_results]

    return {
        "bleu_mean":   float(np.mean(bleu_scores)),
        "bleu_std":    float(np.std(bleu_scores)),
        "rouge1_mean": float(np.mean(rouge1_scores)),
        "rouge1_std":  float(np.std(rouge1_scores)),
        "rouge2_mean": float(np.mean(rouge2_scores)),
        "rouge2_std":  float(np.std(rouge2_scores)),
        "rougeL_mean": float(np.mean(rougeL_scores)),
        "rougeL_std":  float(np.std(rougeL_scores)),
    }

MODEL_DIR  = "unsloth/Llama-3.2-1B-Instruct-unsloth-bnb-4bit"
print(MODEL_DIR)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_DIR,
    max_seq_length=MAX_SEQ_LEN,
    dtype=None,
    load_in_4bit=True,
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.padding_side = "left"

model.eval()
FastLanguageModel.for_inference(model)

results_en_detail = evaluate_model(
    model,
    tokenizer,
    eval_en_clean["test"],
    system_prompt=SYSTEM_PROMPT_EN,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="en",
)

results_id_detail = evaluate_model(
    model,
    tokenizer,
    eval_id_clean["test"],
    system_prompt=SYSTEM_PROMPT_ID,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="id",
)

print("\nFinal Results (test split):")
print(
    f"English (eval_en_clean['test']): "
    f"BLEU = {results_en_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_en_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_en_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_en_detail['rougeL']:.4f}"
)
print(
    f"Indonesian (eval_id_clean['test']): "
    f"BLEU = {results_id_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_id_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_id_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_id_detail['rougeL']:.4f}"
)

unsloth/Llama-3.2-1B-Instruct-unsloth-bnb-4bit
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 34/34 [02:13<00:00,  3.93s/it]



Final Results (test split):
English (eval_en_clean['test']): BLEU = 44.9978, ROUGE-1 = 34.0575, ROUGE-2 = 18.0418, ROUGE-L = 26.1980
Indonesian (eval_id_clean['test']): BLEU = 8.5586, ROUGE-1 = 15.1658, ROUGE-2 = 3.9411, ROUGE-L = 11.4857


## LoRA Baseline

### LoRA-ID

In [64]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [82]:
from sacrebleu.metrics import BLEU
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import random
import transformers
import os
from unsloth import FastLanguageModel
from rouge_score import rouge_scorer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    transformers.set_seed(seed)

# System prompt per bahasa
SYSTEM_PROMPT_ID = (
    "Anda adalah asisten AI yang membantu pengguna dalam bahasa Indonesia. "
)

SYSTEM_PROMPT_EN = (
    "You are a helpful AI assistant that communicates in English. "
)

MAX_SEQ_LEN      = 1024
MAX_NEW_TOKENS   = 128
MAX_INPUT_LENGTH = MAX_SEQ_LEN - MAX_NEW_TOKENS  # supaya total <= 1024

def make_collate_fn(tokenizer, system_prompt, schema):
    def collate_fn(batch):
        input_id_list = []
        references = []

        for item in batch:
            if schema == "id":
                input_text = item["input"]      # ID: input/output
                reference  = item["output"]
            elif schema == "en":
                input_text = item["question"]   # EN: question/response
                reference  = item["response"]
            else:
                raise ValueError(f"Unknown schema: {schema}")

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": input_text},
            ]

            # 1) Tokenisasi per-sample TANPA padding batch
            ids = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
                truncation=True,
                max_length=MAX_INPUT_LENGTH,
            )[0]  # shape: (seq_len,)

            input_id_list.append(ids)
            references.append(reference)

        # 2) Hitung panjang maksimum di batch
        max_len = max(t.shape[0] for t in input_id_list)
        pad_id  = tokenizer.pad_token_id
        batch_size = len(input_id_list)

        # 3) Siapkan tensor batched dengan LEFT PADDING manual
        input_ids = torch.full(
            (batch_size, max_len),
            pad_id,
            dtype=torch.long,
        )
        attention_mask = torch.zeros(
            (batch_size, max_len),
            dtype=torch.long,
        )

        for i, ids in enumerate(input_id_list):
            seq_len = ids.shape[0]
            # left padding: isi dari kanan
            input_ids[i, -seq_len:] = ids
            attention_mask[i, -seq_len:] = 1

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "references": references,
        }

    return collate_fn

def evaluate_model(model, tokenizer, eval_dataset, system_prompt,
                   batch_size=64, max_allowed_length=24, schema="en"):
    model.eval()
    all_predictions = []
    all_references = []        # untuk BLEU (nested list)
    all_references_text = []   # untuk ROUGE (list string)

    collate_fn = make_collate_fn(tokenizer, system_prompt, schema)

    dataloader = DataLoader(
        eval_dataset,
        num_workers=8,              # 0 dulu biar error kelihatan jelas
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=False,
        pin_memory=True,
    )

    bleu = BLEU()
    pbar = tqdm(dataloader)

    for batch in pbar:
        input_ids = batch["input_ids"].to(DEVICE)          # shape: (B, L)
        attention_mask = batch["attention_mask"].to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_allowed_length,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=False,
                use_cache=True,
            )

        # semua sequence punya panjang awal yang sama: L
        seq_len = input_ids.shape[1]

        predictions = []
        for i, output in enumerate(outputs):
            # ambil token hasil generate saja (setelah input penuh)
            generated_tokens = output[seq_len:]
            decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True)
            predictions.append(decoded)

        all_predictions.extend(predictions)
        all_references.extend([[ref] for ref in batch["references"]])
        all_references_text.extend(batch["references"])

    # ====== BLEU ======
    final_bleu = bleu.corpus_score(all_predictions, all_references)

    # ====== ROUGE-1, ROUGE-2, ROUGE-L ======
    scorer = rouge_scorer.RougeScorer(
        ["rouge1", "rouge2", "rougeL"],
        use_stemmer=True,
    )

    rouge1_list, rouge2_list, rougeL_list = [], [], []

    for pred, ref in zip(all_predictions, all_references_text):
        scores = scorer.score(ref, pred)
        rouge1_list.append(scores["rouge1"].fmeasure)
        rouge2_list.append(scores["rouge2"].fmeasure)
        rougeL_list.append(scores["rougeL"].fmeasure)

    # kali 100 supaya bentuknya persen seperti BLEU
    rouge1 = float(np.mean(rouge1_list) * 100)
    rouge2 = float(np.mean(rouge2_list) * 100)
    rougeL = float(np.mean(rougeL_list) * 100)

    return {
        "bleu": final_bleu.score,
        "rouge1": rouge1,
        "rouge2": rouge2,
        "rougeL": rougeL,
        "predictions": all_predictions,
        "references": all_references,
    }


def evaluate_model_multiple_runs(
    model,
    tokenizer,
    eval_dataset,
    system_prompt,
    n_runs=3,
    batch_size=128,
    max_allowed_length=24,
    schema="en",
):
    all_results = []

    for run in range(n_runs):
        set_seed(42 + run)
        results = evaluate_model(
            model,
            tokenizer,
            eval_dataset,
            system_prompt=system_prompt,
            batch_size=batch_size,
            max_allowed_length=max_allowed_length,
            schema=schema,
        )
        all_results.append(results)

        bleu_scores   = [r["bleu"]   for r in all_results]
        rouge1_scores = [r["rouge1"] for r in all_results]
        rouge2_scores = [r["rouge2"] for r in all_results]
        rougeL_scores = [r["rougeL"] for r in all_results]

        print(
            f"Run {run + 1}/{n_runs} - "
            f"BLEU: {np.mean(bleu_scores):.4f} ± {np.std(bleu_scores):.4f} | "
            f"R1: {np.mean(rouge1_scores):.4f} | "
            f"R2: {np.mean(rouge2_scores):.4f} | "
            f"RL: {np.mean(rougeL_scores):.4f}"
        )

    bleu_scores   = [r["bleu"]   for r in all_results]
    rouge1_scores = [r["rouge1"] for r in all_results]
    rouge2_scores = [r["rouge2"] for r in all_results]
    rougeL_scores = [r["rougeL"] for r in all_results]

    return {
        "bleu_mean":   float(np.mean(bleu_scores)),
        "bleu_std":    float(np.std(bleu_scores)),
        "rouge1_mean": float(np.mean(rouge1_scores)),
        "rouge1_std":  float(np.std(rouge1_scores)),
        "rouge2_mean": float(np.mean(rouge2_scores)),
        "rouge2_std":  float(np.std(rouge2_scores)),
        "rougeL_mean": float(np.mean(rougeL_scores)),
        "rougeL_std":  float(np.std(rougeL_scores)),
    }

MODEL_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-id"
print(MODEL_DIR)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_DIR,
    max_seq_length=MAX_SEQ_LEN,
    dtype=None,
    load_in_4bit=True,
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.padding_side = "left"

model.eval()
FastLanguageModel.for_inference(model)

results_en_detail = evaluate_model(
    model,
    tokenizer,
    eval_en_clean["test"],
    system_prompt=SYSTEM_PROMPT_EN,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="en",
)

results_id_detail = evaluate_model(
    model,
    tokenizer,
    eval_id_clean["test"],
    system_prompt=SYSTEM_PROMPT_ID,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="id",
)

print("\nFinal Results (test split):")
print(
    f"English (eval_en_clean['test']): "
    f"BLEU = {results_en_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_en_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_en_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_en_detail['rougeL']:.4f}"
)
print(
    f"Indonesian (eval_id_clean['test']): "
    f"BLEU = {results_id_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_id_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_id_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_id_detail['rougeL']:.4f}"
)

/content/drive/MyDrive/College/KK/Final Project/model/lora-id
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 34/34 [05:22<00:00,  9.48s/it]



Final Results (test split):
English (eval_en_clean['test']): BLEU = 24.6104, ROUGE-1 = 28.9630, ROUGE-2 = 15.8261, ROUGE-L = 24.0982
Indonesian (eval_id_clean['test']): BLEU = 33.0316, ROUGE-1 = 28.9237, ROUGE-2 = 18.8388, ROUGE-L = 27.2290


### LoRA-EN

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [83]:
from sacrebleu.metrics import BLEU
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import random
import transformers
import os
from unsloth import FastLanguageModel
from rouge_score import rouge_scorer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    transformers.set_seed(seed)

# System prompt per bahasa
SYSTEM_PROMPT_ID = (
    "Anda adalah asisten AI yang membantu pengguna dalam bahasa Indonesia. "
)

SYSTEM_PROMPT_EN = (
    "You are a helpful AI assistant that communicates in English. "
)

MAX_SEQ_LEN      = 1024   # sama dengan di FastLanguageModel.from_pretrained
MAX_NEW_TOKENS   = 128
MAX_INPUT_LENGTH = MAX_SEQ_LEN - MAX_NEW_TOKENS  # supaya total <= 1024

def make_collate_fn(tokenizer, system_prompt, schema):
    def collate_fn(batch):
        input_id_list = []
        references = []

        for item in batch:
            if schema == "id":
                input_text = item["input"]      # ID: input/output
                reference  = item["output"]
            elif schema == "en":
                input_text = item["question"]   # EN: question/response
                reference  = item["response"]
            else:
                raise ValueError(f"Unknown schema: {schema}")

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": input_text},
            ]

            # 1) Tokenisasi per-sample TANPA padding batch
            ids = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
                truncation=True,
                max_length=MAX_INPUT_LENGTH,
            )[0]  # shape: (seq_len,)

            input_id_list.append(ids)
            references.append(reference)

        # 2) Hitung panjang maksimum di batch
        max_len = max(t.shape[0] for t in input_id_list)
        pad_id  = tokenizer.pad_token_id
        batch_size = len(input_id_list)

        # 3) Siapkan tensor batched dengan LEFT PADDING manual
        input_ids = torch.full(
            (batch_size, max_len),
            pad_id,
            dtype=torch.long,
        )
        attention_mask = torch.zeros(
            (batch_size, max_len),
            dtype=torch.long,
        )

        for i, ids in enumerate(input_id_list):
            seq_len = ids.shape[0]
            # left padding: isi dari kanan
            input_ids[i, -seq_len:] = ids
            attention_mask[i, -seq_len:] = 1

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "references": references,
        }

    return collate_fn

def evaluate_model(model, tokenizer, eval_dataset, system_prompt,
                   batch_size=64, max_allowed_length=24, schema="en"):
    model.eval()
    all_predictions = []
    all_references = []        # untuk BLEU (nested list)
    all_references_text = []   # untuk ROUGE (list string)

    collate_fn = make_collate_fn(tokenizer, system_prompt, schema)

    dataloader = DataLoader(
        eval_dataset,
        num_workers=8,              # 0 dulu biar error kelihatan jelas
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=False,
        pin_memory=True,
    )

    bleu = BLEU()
    pbar = tqdm(dataloader)

    for batch in pbar:
        input_ids = batch["input_ids"].to(DEVICE)          # shape: (B, L)
        attention_mask = batch["attention_mask"].to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_allowed_length,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=False,
                use_cache=True,
            )

        # semua sequence punya panjang awal yang sama: L
        seq_len = input_ids.shape[1]

        predictions = []
        for i, output in enumerate(outputs):
            # ambil token hasil generate saja (setelah input penuh)
            generated_tokens = output[seq_len:]
            decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True)
            predictions.append(decoded)

        all_predictions.extend(predictions)
        all_references.extend([[ref] for ref in batch["references"]])
        all_references_text.extend(batch["references"])

    # ====== BLEU ======
    final_bleu = bleu.corpus_score(all_predictions, all_references)

    # ====== ROUGE-1, ROUGE-2, ROUGE-L ======
    scorer = rouge_scorer.RougeScorer(
        ["rouge1", "rouge2", "rougeL"],
        use_stemmer=True,
    )

    rouge1_list, rouge2_list, rougeL_list = [], [], []

    for pred, ref in zip(all_predictions, all_references_text):
        scores = scorer.score(ref, pred)
        rouge1_list.append(scores["rouge1"].fmeasure)
        rouge2_list.append(scores["rouge2"].fmeasure)
        rougeL_list.append(scores["rougeL"].fmeasure)

    # kali 100 supaya bentuknya persen seperti BLEU
    rouge1 = float(np.mean(rouge1_list) * 100)
    rouge2 = float(np.mean(rouge2_list) * 100)
    rougeL = float(np.mean(rougeL_list) * 100)

    return {
        "bleu": final_bleu.score,
        "rouge1": rouge1,
        "rouge2": rouge2,
        "rougeL": rougeL,
        "predictions": all_predictions,
        "references": all_references,
    }


def evaluate_model_multiple_runs(
    model,
    tokenizer,
    eval_dataset,
    system_prompt,
    n_runs=3,
    batch_size=128,
    max_allowed_length=24,
    schema="en",
):
    all_results = []

    for run in range(n_runs):
        set_seed(42 + run)
        results = evaluate_model(
            model,
            tokenizer,
            eval_dataset,
            system_prompt=system_prompt,
            batch_size=batch_size,
            max_allowed_length=max_allowed_length,
            schema=schema,
        )
        all_results.append(results)

        bleu_scores   = [r["bleu"]   for r in all_results]
        rouge1_scores = [r["rouge1"] for r in all_results]
        rouge2_scores = [r["rouge2"] for r in all_results]
        rougeL_scores = [r["rougeL"] for r in all_results]

        print(
            f"Run {run + 1}/{n_runs} - "
            f"BLEU: {np.mean(bleu_scores):.4f} ± {np.std(bleu_scores):.4f} | "
            f"R1: {np.mean(rouge1_scores):.4f} | "
            f"R2: {np.mean(rouge2_scores):.4f} | "
            f"RL: {np.mean(rougeL_scores):.4f}"
        )

    bleu_scores   = [r["bleu"]   for r in all_results]
    rouge1_scores = [r["rouge1"] for r in all_results]
    rouge2_scores = [r["rouge2"] for r in all_results]
    rougeL_scores = [r["rougeL"] for r in all_results]

    return {
        "bleu_mean":   float(np.mean(bleu_scores)),
        "bleu_std":    float(np.std(bleu_scores)),
        "rouge1_mean": float(np.mean(rouge1_scores)),
        "rouge1_std":  float(np.std(rouge1_scores)),
        "rouge2_mean": float(np.mean(rouge2_scores)),
        "rouge2_std":  float(np.std(rouge2_scores)),
        "rougeL_mean": float(np.mean(rougeL_scores)),
        "rougeL_std":  float(np.std(rougeL_scores)),
    }

MODEL_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-en"
print(MODEL_DIR)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_DIR,
    max_seq_length=MAX_SEQ_LEN,
    dtype=None,
    load_in_4bit=True,
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.padding_side = "left"

model.eval()
FastLanguageModel.for_inference(model)

results_en_detail = evaluate_model(
    model,
    tokenizer,
    eval_en_clean["test"],
    system_prompt=SYSTEM_PROMPT_EN,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="en",
)

results_id_detail = evaluate_model(
    model,
    tokenizer,
    eval_id_clean["test"],
    system_prompt=SYSTEM_PROMPT_ID,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="id",
)

print("\nFinal Results (test split):")
print(
    f"English (eval_en_clean['test']): "
    f"BLEU = {results_en_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_en_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_en_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_en_detail['rougeL']:.4f}"
)
print(
    f"Indonesian (eval_id_clean['test']): "
    f"BLEU = {results_id_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_id_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_id_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_id_detail['rougeL']:.4f}"
)

/content/drive/MyDrive/College/KK/Final Project/model/lora-en
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 34/34 [05:21<00:00,  9.46s/it]



Final Results (test split):
English (eval_en_clean['test']): BLEU = 43.9476, ROUGE-1 = 38.1236, ROUGE-2 = 22.6967, ROUGE-L = 30.5489
Indonesian (eval_id_clean['test']): BLEU = 22.0896, ROUGE-1 = 14.6993, ROUGE-2 = 3.6265, ROUGE-L = 10.8722


## LoRA-Soups Baseline

In [ ]:
# Configuration
BASE_MODEL = "unsloth/Llama-3.2-1B-Instruct-unsloth-bnb-4bit"

LORA1_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-id"
LORA2_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-en"

LORA_SOUP_DIR = "/content/drive/MyDrive/College/KK/Final Project/model/lora_soup_base_05_05"

CAT_WEIGHTS = [0.5, 0.5]

MAX_SEQ_LEN  = 2048
LOAD_IN_4BIT = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

Device: cuda


### Building LoRA-Soups (CAT)

In [ ]:
def show_adapter_cfg(path):
    with open(os.path.join(path, "adapter_config.json"), "r") as f:
        cfg = json.load(f)
    print("===", path, "===")
    print("r           :", cfg["r"])
    print("lora_alpha  :", cfg.get("lora_alpha"))
    print("target_modules:", cfg.get("target_modules"))

show_adapter_cfg(LORA1_DIR)
show_adapter_cfg(LORA2_DIR)

=== /content/drive/MyDrive/College/KK/Final Project/model/lora-id ===
r           : 16
lora_alpha  : 16
target_modules: ['k_proj', 'o_proj', 'up_proj', 'down_proj', 'v_proj', 'gate_proj', 'q_proj']
=== /content/drive/MyDrive/College/KK/Final Project/model/lora-en ===
r           : 16
lora_alpha  : 16
target_modules: ['q_proj', 'o_proj', 'down_proj', 'v_proj', 'up_proj', 'k_proj', 'gate_proj']


In [ ]:
import os, json, shutil
from typing import List, Optional

import torch
from safetensors.torch import load_file, save_file

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def build_lora_soup(
    lora_paths: List[str],
    weights: Optional[List[float]],
    out_path: str,
    copy_tokenizer: bool = True,
):
    """
    Menggabungkan beberapa LoRA menjadi 1 adapter LoRA-Soups
    dengan cara *weighted average* (model soup) atas seluruh
    bobot float di state_dict.

    lora_paths : list path adapter LoRA sumber
    weights    : bobot soup, akan dinormalisasi; jika None -> uniform
    out_path   : folder output adapter baru
    """
    assert len(lora_paths) >= 2, "Minimal 2 LoRA untuk LoRA-Soups"

    # jika weights tidak diberikan -> uniform
    if weights is None:
        w = torch.ones(len(lora_paths), dtype=torch.float32)
    else:
        assert len(weights) == len(lora_paths), "Panjang weights harus = jumlah LoRA"
        w = torch.tensor(weights, dtype=torch.float32)

    # normalisasi bobot
    w = w / w.sum()

    os.makedirs(out_path, exist_ok=True)

    # 1. baca config dari adapter pertama
    cfg_path0 = os.path.join(lora_paths[0], "adapter_config.json")
    with open(cfg_path0, "r") as f:
        cfg0 = json.load(f)

    base_r      = cfg0["r"]
    base_alpha  = cfg0.get("lora_alpha", None)
    base_target = set(cfg0.get("target_modules", []))

    # cek konsistensi adapter lain
    for p in lora_paths[1:]:
        with open(os.path.join(p, "adapter_config.json"), "r") as f:
            cfgi = json.load(f)
        assert cfgi["r"] == base_r, f"Rank LoRA beda di {p}"
        if "lora_alpha" in cfgi and base_alpha is not None:
            assert cfgi["lora_alpha"] == base_alpha, f"lora_alpha beda di {p}"
        if "target_modules" in cfgi and base_target:
            assert set(cfgi["target_modules"]) == base_target, f"target_modules beda di {p}"

    # 2. config baru untuk SOUP:
    #    rank r & lora_alpha TIDAK berubah, hanya bobot yang di-average
    cfg_soup = cfg0.copy()
    if "target_modules" in cfg_soup:
        cfg_soup["target_modules"] = sorted(cfg_soup["target_modules"])

    with open(os.path.join(out_path, "adapter_config.json"), "w") as f:
        json.dump(cfg_soup, f, indent=2)

    # 3. load semua state dict LoRA
    state_dicts = [load_file(os.path.join(p, "adapter_model.safetensors")) for p in lora_paths]

    # opsional: pastikan key sama semua
    keys0 = set(state_dicts[0].keys())
    for i, sd in enumerate(state_dicts[1:], start=1):
        ki = set(sd.keys())
        assert keys0 == ki, f"Kumpulan key state_dict LoRA ke-{i} beda; cek adapter: {lora_paths[i]}"

    # 4. build state dict baru dengan weighted average
    new_state = {}
    float_dtypes = {
        torch.float16,
        torch.bfloat16,
        torch.float32,
        torch.float64,
    }

    for key in state_dicts[0].keys():
        tensor0 = state_dicts[0][key]

        # Kalau tensor floating -> soup (weighted average)
        if isinstance(tensor0, torch.Tensor) and tensor0.dtype in float_dtypes:
            acc = torch.zeros_like(tensor0)
            for wi, sd in zip(w, state_dicts):
                acc = acc + wi * sd[key]
            new_state[key] = acc
        else:
            # Non-float (misal buffer int, dsb) -> ambil dari adapter pertama saja
            new_state[key] = tensor0

    # 5. simpan
    save_file(new_state, os.path.join(out_path, "adapter_model.safetensors"))

    # 6. copy tokenizer & template dari adapter pertama (opsional)
    if copy_tokenizer:
        extra_files = [
            "tokenizer.json",
            "tokenizer_config.json",
            "special_tokens_map.json",
            "chat_template.jinja",
            "README.md",
        ]
        for fname in extra_files:
            src = os.path.join(lora_paths[0], fname)
            if os.path.exists(src):
                shutil.copy(src, os.path.join(out_path, fname))

    print(f"[OK] LoRA-Soups disimpan di: {out_path}")

In [ ]:
build_lora_soup(
    lora_paths=[LORA1_DIR, LORA2_DIR],
    weights=CAT_WEIGHTS,
    out_path=LORA_SOUP_DIR,
    copy_tokenizer=True,
)

[OK] LoRA-Soups disimpan di: /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_base_05_05


### Evaluation

In [84]:
from sacrebleu.metrics import BLEU
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import random
import transformers
import os
from unsloth import FastLanguageModel
from rouge_score import rouge_scorer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    transformers.set_seed(seed)

# System prompt per bahasa
SYSTEM_PROMPT_ID = (
    "Anda adalah asisten AI yang membantu pengguna dalam bahasa Indonesia. "
)

SYSTEM_PROMPT_EN = (
    "You are a helpful AI assistant that communicates in English. "
)

MAX_SEQ_LEN      = 1024   # sama dengan di FastLanguageModel.from_pretrained
MAX_NEW_TOKENS   = 128
MAX_INPUT_LENGTH = MAX_SEQ_LEN - MAX_NEW_TOKENS  # supaya total <= 1024

def make_collate_fn(tokenizer, system_prompt, schema):
    def collate_fn(batch):
        input_id_list = []
        references = []

        for item in batch:
            if schema == "id":
                input_text = item["input"]      # ID: input/output
                reference  = item["output"]
            elif schema == "en":
                input_text = item["question"]   # EN: question/response
                reference  = item["response"]
            else:
                raise ValueError(f"Unknown schema: {schema}")

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": input_text},
            ]

            # 1) Tokenisasi per-sample TANPA padding batch
            ids = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
                truncation=True,
                max_length=MAX_INPUT_LENGTH,
            )[0]  # shape: (seq_len,)

            input_id_list.append(ids)
            references.append(reference)

        # 2) Hitung panjang maksimum di batch
        max_len = max(t.shape[0] for t in input_id_list)
        pad_id  = tokenizer.pad_token_id
        batch_size = len(input_id_list)

        # 3) Siapkan tensor batched dengan LEFT PADDING manual
        input_ids = torch.full(
            (batch_size, max_len),
            pad_id,
            dtype=torch.long,
        )
        attention_mask = torch.zeros(
            (batch_size, max_len),
            dtype=torch.long,
        )

        for i, ids in enumerate(input_id_list):
            seq_len = ids.shape[0]
            # left padding: isi dari kanan
            input_ids[i, -seq_len:] = ids
            attention_mask[i, -seq_len:] = 1

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "references": references,
        }

    return collate_fn

def evaluate_model(model, tokenizer, eval_dataset, system_prompt,
                   batch_size=64, max_allowed_length=24, schema="en"):
    model.eval()
    all_predictions = []
    all_references = []        # untuk BLEU (nested list)
    all_references_text = []   # untuk ROUGE (list string)

    collate_fn = make_collate_fn(tokenizer, system_prompt, schema)

    dataloader = DataLoader(
        eval_dataset,
        num_workers=8,              # 0 dulu biar error kelihatan jelas
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=False,
        pin_memory=True,
    )

    bleu = BLEU()
    pbar = tqdm(dataloader)

    for batch in pbar:
        input_ids = batch["input_ids"].to(DEVICE)          # shape: (B, L)
        attention_mask = batch["attention_mask"].to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_allowed_length,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=False,
                use_cache=True,
            )

        # semua sequence punya panjang awal yang sama: L
        seq_len = input_ids.shape[1]

        predictions = []
        for i, output in enumerate(outputs):
            # ambil token hasil generate saja (setelah input penuh)
            generated_tokens = output[seq_len:]
            decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True)
            predictions.append(decoded)

        all_predictions.extend(predictions)
        all_references.extend([[ref] for ref in batch["references"]])
        all_references_text.extend(batch["references"])

    # ====== BLEU ======
    final_bleu = bleu.corpus_score(all_predictions, all_references)

    # ====== ROUGE-1, ROUGE-2, ROUGE-L ======
    scorer = rouge_scorer.RougeScorer(
        ["rouge1", "rouge2", "rougeL"],
        use_stemmer=True,
    )

    rouge1_list, rouge2_list, rougeL_list = [], [], []

    for pred, ref in zip(all_predictions, all_references_text):
        scores = scorer.score(ref, pred)
        rouge1_list.append(scores["rouge1"].fmeasure)
        rouge2_list.append(scores["rouge2"].fmeasure)
        rougeL_list.append(scores["rougeL"].fmeasure)

    # kali 100 supaya bentuknya persen seperti BLEU
    rouge1 = float(np.mean(rouge1_list) * 100)
    rouge2 = float(np.mean(rouge2_list) * 100)
    rougeL = float(np.mean(rougeL_list) * 100)

    return {
        "bleu": final_bleu.score,
        "rouge1": rouge1,
        "rouge2": rouge2,
        "rougeL": rougeL,
        "predictions": all_predictions,
        "references": all_references,
    }


def evaluate_model_multiple_runs(
    model,
    tokenizer,
    eval_dataset,
    system_prompt,
    n_runs=3,
    batch_size=128,
    max_allowed_length=24,
    schema="en",
):
    all_results = []

    for run in range(n_runs):
        set_seed(42 + run)
        results = evaluate_model(
            model,
            tokenizer,
            eval_dataset,
            system_prompt=system_prompt,
            batch_size=batch_size,
            max_allowed_length=max_allowed_length,
            schema=schema,
        )
        all_results.append(results)

        bleu_scores   = [r["bleu"]   for r in all_results]
        rouge1_scores = [r["rouge1"] for r in all_results]
        rouge2_scores = [r["rouge2"] for r in all_results]
        rougeL_scores = [r["rougeL"] for r in all_results]

        print(
            f"Run {run + 1}/{n_runs} - "
            f"BLEU: {np.mean(bleu_scores):.4f} ± {np.std(bleu_scores):.4f} | "
            f"R1: {np.mean(rouge1_scores):.4f} | "
            f"R2: {np.mean(rouge2_scores):.4f} | "
            f"RL: {np.mean(rougeL_scores):.4f}"
        )

    bleu_scores   = [r["bleu"]   for r in all_results]
    rouge1_scores = [r["rouge1"] for r in all_results]
    rouge2_scores = [r["rouge2"] for r in all_results]
    rougeL_scores = [r["rougeL"] for r in all_results]

    return {
        "bleu_mean":   float(np.mean(bleu_scores)),
        "bleu_std":    float(np.std(bleu_scores)),
        "rouge1_mean": float(np.mean(rouge1_scores)),
        "rouge1_std":  float(np.std(rouge1_scores)),
        "rouge2_mean": float(np.mean(rouge2_scores)),
        "rouge2_std":  float(np.std(rouge2_scores)),
        "rougeL_mean": float(np.mean(rougeL_scores)),
        "rougeL_std":  float(np.std(rougeL_scores)),
    }

MODEL_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora_soup_base_05_05"
print(MODEL_DIR)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_DIR,
    max_seq_length=MAX_SEQ_LEN,
    dtype=None,
    load_in_4bit=True,
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.padding_side = "left"

model.eval()
FastLanguageModel.for_inference(model)

results_en_detail = evaluate_model(
    model,
    tokenizer,
    eval_en_clean["test"],
    system_prompt=SYSTEM_PROMPT_EN,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="en",
)

results_id_detail = evaluate_model(
    model,
    tokenizer,
    eval_id_clean["test"],
    system_prompt=SYSTEM_PROMPT_ID,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="id",
)

print("\nFinal Results (test split):")
print(
    f"English (eval_en_clean['test']): "
    f"BLEU = {results_en_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_en_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_en_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_en_detail['rougeL']:.4f}"
)
print(
    f"Indonesian (eval_id_clean['test']): "
    f"BLEU = {results_id_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_id_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_id_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_id_detail['rougeL']:.4f}"
)

/content/drive/MyDrive/College/KK/Final Project/model/lora_soup_base_05_05
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 34/34 [05:27<00:00,  9.62s/it]



Final Results (test split):
English (eval_en_clean['test']): BLEU = 51.5411, ROUGE-1 = 35.3187, ROUGE-2 = 21.6106, ROUGE-L = 29.6817
Indonesian (eval_id_clean['test']): BLEU = 15.7930, ROUGE-1 = 16.0890, ROUGE-2 = 4.9820, ROUGE-L = 13.2332


## LoRA-Soups (GA)

### Search for the best weight

In [ ]:
N_HEAD = 10
N_MID = 30
N_TAIL = 10
N = N_HEAD + N_MID + N_TAIL

ds_id = eval_id_clean["test"]
ds_en = eval_en_clean["test"]

n_total = len(ds_id)
assert len(ds_en) == n_total, "Panjang split ID dan EN harus sama"
assert n_total >= N, "Dataset terlalu kecil untuk ambil 50 sampel"

head_indices = list(range(0, N_HEAD))

mid_start = (n_total - N_MID) // 2
mid_indices = list(range(mid_start, mid_start + N_MID))

tail_indices = list(range(n_total - N_TAIL, n_total))

indices = head_indices + mid_indices + tail_indices

test_id_samples = ds_id.select(indices)
test_en_samples = ds_en.select(indices)

display(test_id_samples)
display(test_en_samples)

Dataset({
    features: ['dataset_name', 'subset_name', 'prompt_id', 'template_name', 'dataset_key', 'input', 'output'],
    num_rows: 50
})

Dataset({
    features: ['id', 'system_prompt', 'question', 'response'],
    num_rows: 50
})

In [ ]:
from safetensors.torch import load_file, save_file
import shutil

LORA1_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-id"
LORA2_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-en"

LORA_SOUP_TMP_DIR = "/content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_tmp"

In [ ]:
FLOAT_DTYPES = {
    torch.float16,
    torch.bfloat16,
    torch.float32,
    torch.float64,
}


def load_two_lora_state_dicts(lora_dir_1, lora_dir_2):
    """
    Load state_dict dari 2 adapter LoRA (safetensors).
    Pastikan kedua adapter dilatih dari base dan config yang sama.
    """
    sd1 = load_file(os.path.join(lora_dir_1, "adapter_model.safetensors"))
    sd2 = load_file(os.path.join(lora_dir_2, "adapter_model.safetensors"))

    keys1 = set(sd1.keys())
    keys2 = set(sd2.keys())
    if keys1 != keys2:
        raise ValueError("Key di adapter LoRA 1 dan 2 tidak sama. "
                         "Pastikan keduanya dilatih dari base + config yang sama.")

    return sd1, sd2


def prepare_soup_dir(template_dir, soup_dir):
    """
    Membuat folder soup sekali saja dengan cara copy seluruh isi template_dir
    (misal dari LORA1_DIR) ke soup_dir.
    Nanti setiap kali GA update bobot, kita hanya overwrite adapter_model.safetensors di soup_dir.
    """
    if os.path.exists(soup_dir):
        shutil.rmtree(soup_dir)
    shutil.copytree(template_dir, soup_dir)
    print(f"[INFO] Folder soup dibuat dari template: {template_dir} -> {soup_dir}")


def write_soup_state_dict(sd1, sd2, alpha, out_dir):
    """
    Membuat LoRA-Soup dari 2 adapter:
        W_soup = alpha * W1 + (1 - alpha) * W2

    alpha di [0, 1]. Hasilnya disimpan ke adapter_model.safetensors di out_dir.
    """
    alpha = float(alpha)
    if not (0.0 <= alpha <= 1.0):
        raise ValueError("alpha harus di antara 0 dan 1")

    w1 = alpha
    w2 = 1.0 - alpha

    new_state = {}
    for k in sd1.keys():
        t1 = sd1[k]
        t2 = sd2[k]

        if isinstance(t1, torch.Tensor) and t1.dtype in FLOAT_DTYPES:
            new_state[k] = w1 * t1 + w2 * t2
        else:
            new_state[k] = t1

    out_path = os.path.join(out_dir, "adapter_model.safetensors")
    save_file(new_state, out_path)
    print(f"[DEBUG] Soup weights disimpan ke {out_path}")

In [ ]:
# Sanity Check LoRA
lora_sd1, lora_sd2 = load_two_lora_state_dicts(LORA1_DIR, LORA2_DIR)
prepare_soup_dir(LORA1_DIR, LORA_SOUP_TMP_DIR)

[INFO] Folder soup dibuat dari template: /content/drive/MyDrive/College/KK/Final Project/model/lora-id -> /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_tmp


In [ ]:
from sacrebleu.metrics import BLEU
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import random
import transformers
import os
from unsloth import FastLanguageModel
from rouge_score import rouge_scorer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    transformers.set_seed(seed)

SYSTEM_PROMPT_ID = (
    "Anda adalah asisten AI yang membantu pengguna dalam bahasa Indonesia. "
)

SYSTEM_PROMPT_EN = (
    "You are a helpful AI assistant that communicates in English. "
)

MAX_SEQ_LEN      = 1024
MAX_NEW_TOKENS   = 128
MAX_INPUT_LENGTH = MAX_SEQ_LEN - MAX_NEW_TOKENS

def make_collate_fn(tokenizer, system_prompt, schema):
    def collate_fn(batch):
        input_id_list = []
        references = []

        for item in batch:
            if schema == "id":
                input_text = item["input"]      # ID: input/output
                reference  = item["output"]
            elif schema == "en":
                input_text = item["question"]   # EN: question/response
                reference  = item["response"]
            else:
                raise ValueError(f"Unknown schema: {schema}")

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": input_text},
            ]

            # 1) Tokenisasi per-sample TANPA padding batch
            ids = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
                truncation=True,
                max_length=MAX_INPUT_LENGTH,
            )[0]  # shape: (seq_len,)

            input_id_list.append(ids)
            references.append(reference)

        # 2) Hitung panjang maksimum di batch
        max_len = max(t.shape[0] for t in input_id_list)
        pad_id  = tokenizer.pad_token_id
        batch_size = len(input_id_list)

        # 3) Siapkan tensor batched dengan LEFT PADDING manual
        input_ids = torch.full(
            (batch_size, max_len),
            pad_id,
            dtype=torch.long,
        )
        attention_mask = torch.zeros(
            (batch_size, max_len),
            dtype=torch.long,
        )

        for i, ids in enumerate(input_id_list):
            seq_len = ids.shape[0]
            # left padding: isi dari kanan
            input_ids[i, -seq_len:] = ids
            attention_mask[i, -seq_len:] = 1

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "references": references,
        }

    return collate_fn

def evaluate_model(model, tokenizer, eval_dataset, system_prompt,
                   batch_size=64, max_allowed_length=24, schema="en"):
    model.eval()
    all_predictions = []
    all_references = []        # untuk BLEU (nested list)
    all_references_text = []   # untuk ROUGE (list string)

    collate_fn = make_collate_fn(tokenizer, system_prompt, schema)

    dataloader = DataLoader(
        eval_dataset,
        num_workers=8,              # 0 dulu biar error kelihatan jelas
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=False,
        pin_memory=True,
    )

    bleu = BLEU()
    pbar = tqdm(dataloader)

    for batch in pbar:
        input_ids = batch["input_ids"].to(DEVICE)          # shape: (B, L)
        attention_mask = batch["attention_mask"].to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_allowed_length,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=False,
                use_cache=True,
            )

        # semua sequence punya panjang awal yang sama: L
        seq_len = input_ids.shape[1]

        predictions = []
        for i, output in enumerate(outputs):
            # ambil token hasil generate saja (setelah input penuh)
            generated_tokens = output[seq_len:]
            decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True)
            predictions.append(decoded)

        all_predictions.extend(predictions)
        all_references.extend([[ref] for ref in batch["references"]])
        all_references_text.extend(batch["references"])

    # ====== BLEU ======
    final_bleu = bleu.corpus_score(all_predictions, all_references)

    # ====== ROUGE-1, ROUGE-2, ROUGE-L ======
    scorer = rouge_scorer.RougeScorer(
        ["rouge1", "rouge2", "rougeL"],
        use_stemmer=True,
    )

    rouge1_list, rouge2_list, rougeL_list = [], [], []

    for pred, ref in zip(all_predictions, all_references_text):
        scores = scorer.score(ref, pred)
        rouge1_list.append(scores["rouge1"].fmeasure)
        rouge2_list.append(scores["rouge2"].fmeasure)
        rougeL_list.append(scores["rougeL"].fmeasure)

    # kali 100 supaya bentuknya persen seperti BLEU
    rouge1 = float(np.mean(rouge1_list) * 100)
    rouge2 = float(np.mean(rouge2_list) * 100)
    rougeL = float(np.mean(rougeL_list) * 100)

    return {
        "bleu": final_bleu.score,
        "rouge1": rouge1,
        "rouge2": rouge2,
        "rougeL": rougeL,
        "predictions": all_predictions,
        "references": all_references,
    }


def evaluate_model_multiple_runs(
    model,
    tokenizer,
    eval_dataset,
    system_prompt,
    n_runs=3,
    batch_size=128,
    max_allowed_length=24,
    schema="en",
):
    all_results = []

    for run in range(n_runs):
        set_seed(42 + run)
        results = evaluate_model(
            model,
            tokenizer,
            eval_dataset,
            system_prompt=system_prompt,
            batch_size=batch_size,
            max_allowed_length=max_allowed_length,
            schema=schema,
        )
        all_results.append(results)

        bleu_scores   = [r["bleu"]   for r in all_results]
        rouge1_scores = [r["rouge1"] for r in all_results]
        rouge2_scores = [r["rouge2"] for r in all_results]
        rougeL_scores = [r["rougeL"] for r in all_results]

        print(
            f"Run {run + 1}/{n_runs} - "
            f"BLEU: {np.mean(bleu_scores):.4f} ± {np.std(bleu_scores):.4f} | "
            f"R1: {np.mean(rouge1_scores):.4f} | "
            f"R2: {np.mean(rouge2_scores):.4f} | "
            f"RL: {np.mean(rougeL_scores):.4f}"
        )

    bleu_scores   = [r["bleu"]   for r in all_results]
    rouge1_scores = [r["rouge1"] for r in all_results]
    rouge2_scores = [r["rouge2"] for r in all_results]
    rougeL_scores = [r["rougeL"] for r in all_results]

    return {
        "bleu_mean":   float(np.mean(bleu_scores)),
        "bleu_std":    float(np.std(bleu_scores)),
        "rouge1_mean": float(np.mean(rouge1_scores)),
        "rouge1_std":  float(np.std(rouge1_scores)),
        "rouge2_mean": float(np.mean(rouge2_scores)),
        "rouge2_std":  float(np.std(rouge2_scores)),
        "rougeL_mean": float(np.mean(rougeL_scores)),
        "rougeL_std":  float(np.std(rougeL_scores)),
    }

def evaluate_alpha(alpha, eval_en_dataset, eval_id_dataset, batch_size=75, verbose=True, weight_id=0.5):
    """
    Evaluasi 1 nilai alpha:
        w1 = alpha, w2 = 1 - alpha
    Fitness = (1 - weight_id)*BLEU_EN + weight_id*BLEU_ID
    """

    # 1. Tulis LoRA-Soup ke LORA_SOUP_TMP_DIR
    write_soup_state_dict(lora_sd1, lora_sd2, alpha, LORA_SOUP_TMP_DIR)

    # 2. Bersihkan GPU sebelum load model baru
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # 3. Load model + tokenizer dari adapter soup
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=LORA_SOUP_TMP_DIR,
        max_seq_length=MAX_SEQ_LEN,
        dtype=None,
        load_in_4bit=True,
    )

    # Set pad token dan padding side sesuai evaluasi sebelumnya
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id
    tokenizer.padding_side = "left"

    model.to(DEVICE)
    model.eval()
    FastLanguageModel.for_inference(model)

    results_en = evaluate_model(
        model,
        tokenizer,
        eval_en_dataset,
        system_prompt=SYSTEM_PROMPT_EN,
        batch_size=batch_size,
        max_allowed_length=MAX_NEW_TOKENS,
        schema="en",
    )

    results_id = evaluate_model(
        model,
        tokenizer,
        eval_id_dataset,
        system_prompt=SYSTEM_PROMPT_ID,
        batch_size=batch_size,
        max_allowed_length=MAX_NEW_TOKENS,
        schema="id",
    )

    bleu_en = results_en["bleu"]
    bleu_id = results_id["bleu"]

    fitness = (1.0 - weight_id) * bleu_en + weight_id * bleu_id

    if verbose:
        print(
            f"[alpha={alpha:.4f}] "
            f"BLEU_EN={bleu_en:.4f}, BLEU_ID={bleu_id:.4f}, "
            f"Fitness={fitness:.4f}"
        )

    del model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return fitness, bleu_en, bleu_id

In [ ]:
# Quick Test
fit_05, bleu_en_05, bleu_id_05 = evaluate_alpha(
    alpha=0.5,
    eval_en_dataset=test_en_samples,
    eval_id_dataset=test_id_samples,
)
print("Fitness alpha=0.5:", fit_05)

[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.80s/it]


[alpha=0.5000] BLEU_EN=27.9623, BLEU_ID=7.7550, Fitness=17.8587
Fitness alpha=0.5: 17.858654659790623


In [ ]:
# GA Hyperparameters
POP_SIZE       = 8     # jumlah individu per generasi
N_GENERATIONS  = 6     # jumlah generasi
ELITE_FRAC     = 0.25  # proporsi elit yang langsung lolos
MUTATION_STD   = 0.10  # standar deviasi noise mutasi

CROSSOVER_RATE  = 0.9
MUTATION_RATE   = 0.1
TOURNAMENT_SIZE = 3


def init_population(pop_size):
    """
    Inisialisasi populasi alpha secara uniform di [0, 1].
    """
    return np.random.rand(pop_size).astype(np.float32)


def mutate_alpha(alpha, sigma=MUTATION_STD):
    """
    Mutasi alpha dengan Gaussian noise, lalu di-clamp ke [0, 1].
    """
    new_alpha = alpha + np.random.normal(0.0, sigma)
    new_alpha = float(np.clip(new_alpha, 0.0, 1.0))
    return new_alpha


def crossover_alpha(alpha1, alpha2):
    """
    Crossover sederhana: convex combination antara dua parent.
    (Real-coded GA / arithmetic crossover)
    """
    t = np.random.rand()
    child = float(t * alpha1 + (1.0 - t) * alpha2)
    return child


def tournament_select(population, fitnesses, k=TOURNAMENT_SIZE):
    n = len(population)
    k = min(k, n)  # biar gak lebih besar dari ukuran populasi
    idxs = np.random.choice(n, size=k, replace=False)
    best_local_idx = idxs[np.argmax(fitnesses[idxs])]
    return float(population[best_local_idx])


def reproduce_offspring(population, fitnesses):
    """
    Buat satu child:
      - pilih 2 parent dengan tournament selection
      - crossover dengan probabilitas CROSSOVER_RATE
      - mutasi dengan probabilitas MUTATION_RATE
    """
    p1 = tournament_select(population, fitnesses)
    p2 = tournament_select(population, fitnesses)

    # Crossover
    if np.random.rand() < CROSSOVER_RATE:
        child = crossover_alpha(p1, p2)
    else:
        child = p1 if np.random.rand() < 0.5 else p2

    # Mutasi
    if np.random.rand() < MUTATION_RATE:
        child = mutate_alpha(child)

    return float(child)


def run_ga_for_lora_soup(
    eval_en_dataset,
    eval_id_dataset,
    pop_size=POP_SIZE,
    n_generations=N_GENERATIONS,
    elite_frac=ELITE_FRAC,
):
    """
    Jalankan GA untuk mencari alpha terbaik.
    Return:
        best_alpha, best_fitness, history (list per generasi)
    """
    set_seed(123)  # biar cukup reproducible

    population = init_population(pop_size)
    best_alpha = None
    best_fitness = -1e9
    history = []

    for gen in range(n_generations):
        print(f"\n=== Generasi {gen + 1}/{n_generations}")
        fitnesses = []
        per_individual_stats = []

        # 1. Evaluasi semua individu di populasi
        for idx, alpha in enumerate(population):
            print(f"  Individu {idx + 1}/{len(population)}: alpha={alpha:.4f}")
            fitness, bleu_en, bleu_id = evaluate_alpha(
                alpha=alpha,
                eval_en_dataset=eval_en_dataset,
                eval_id_dataset=eval_id_dataset,
                batch_size=80,
                verbose=True,
                weight_id=0.5,  # 0.5 = EN & ID sama penting
            )
            fitnesses.append(fitness)
            per_individual_stats.append({
                "alpha": float(alpha),
                "fitness": float(fitness),
                "bleu_en": float(bleu_en),
                "bleu_id": float(bleu_id),
            })

        fitnesses = np.array(fitnesses, dtype=np.float32)

        # 2. Cari terbaik di generasi ini
        gen_best_idx = int(fitnesses.argmax())
        gen_best_alpha = float(population[gen_best_idx])
        gen_best_fitness = float(fitnesses[gen_best_idx])

        if gen_best_fitness > best_fitness:
            best_fitness = gen_best_fitness
            best_alpha = gen_best_alpha

        print(
            f" >> Best generasi {gen + 1}: alpha={gen_best_alpha:.4f}, "
            f"fitness={gen_best_fitness:.4f}"
        )
        print(
            f" >> Global best sejauh ini: alpha={best_alpha:.4f}, "
            f"fitness={best_fitness:.4f}"
        )

        history.append({
            "generation": gen + 1,
            "gen_best_alpha": gen_best_alpha,
            "gen_best_fitness": gen_best_fitness,
            "individuals": per_individual_stats,
        })

        # 3. Seleksi elit
        n_elite = max(1, int(elite_frac * pop_size))
        elite_indices = fitnesses.argsort()[::-1][:n_elite]
        new_population = [float(population[i]) for i in elite_indices]

        # 4. Isi sisa populasi dengan offspring via seleksi + crossover + mutasi
        while len(new_population) < pop_size:
            child = reproduce_offspring(population, fitnesses)
            new_population.append(child)

        population = np.array(new_population, dtype=np.float32)

    print("\n=== GA selesai")
    print(f"Bobot terbaik (alpha) = {best_alpha:.4f}")
    print(f" -> w_LoRA1 = {best_alpha:.4f}")
    print(f" -> w_LoRA2 = {1.0 - best_alpha:.4f}")
    print(f"Fitness BLEU gabungan terbaik = {best_fitness:.4f}")

    return best_alpha, best_fitness, history

In [ ]:
import psutil, os, time

process = psutil.Process(os.getpid())

if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.empty_cache()

start_gpu_mem = torch.cuda.memory_allocated()
start_ram = process.memory_info().rss
start_time = time.perf_counter()

best_alpha, best_fitness, ga_history = run_ga_for_lora_soup(
    eval_en_dataset=test_en_samples,
    eval_id_dataset=test_id_samples,
    pop_size=8,
    n_generations=6,
    elite_frac=0.25,
)

end_time = time.perf_counter()
end_ram = process.memory_info().rss
end_gpu_mem = torch.cuda.memory_allocated()
peak_gpu_mem = torch.cuda.max_memory_allocated()

elapsed = end_time - start_time

print("\n=== HASIL AKHIR GA ===")
print(f"alpha terbaik    : {best_alpha:.4f}")
print(f"w_LoRA1 (adapter1): {best_alpha:.4f}")
print(f"w_LoRA2 (adapter2): {1.0 - best_alpha:.4f}")
print(f"fitness (BLEU mix): {best_fitness:.4f}")
print(f"Total waktu GA     : {elapsed:.2f} detik")

print("\n=== MEMORI GPU ===")
print(f"Memori awal  : {start_gpu_mem / (1024**2):.2f} MB")
print(f"Memori akhir : {end_gpu_mem / (1024**2):.2f} MB")
print(f"Peak usage   : {peak_gpu_mem / (1024**2):.2f} MB")

print("\n=== MEMORI RAM ===")
print(f"RAM awal  : {start_ram / (1024**2):.2f} MB")
print(f"RAM akhir : {end_ram / (1024**2):.2f} MB")
print(f"Δ RAM     : {(end_ram - start_ram) / (1024**2):.2f} MB")


=== Generasi 1/6
  Individu 1/8: alpha=0.6965
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.68s/it]


[alpha=0.6965] BLEU_EN=12.4092, BLEU_ID=7.2880, Fitness=9.8486
  Individu 2/8: alpha=0.2861
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.78s/it]


[alpha=0.2861] BLEU_EN=19.0632, BLEU_ID=4.7788, Fitness=11.9210
  Individu 3/8: alpha=0.2269
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

[alpha=0.5513] BLEU_EN=30.2178, BLEU_ID=13.4851, Fitness=21.8515
  Individu 7/8: alpha=0.5513
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.88s/it]


[alpha=0.5513] BLEU_EN=30.2178, BLEU_ID=13.4851, Fitness=21.8515
  Individu 8/8: alpha=0.5513
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.86s/it]


[alpha=0.5513] BLEU_EN=30.2178, BLEU_ID=13.4851, Fitness=21.8515
 >> Best generasi 6: alpha=0.5513, fitness=21.8515
 >> Global best sejauh ini: alpha=0.5513, fitness=21.8515

=== GA selesai
Bobot terbaik (alpha) = 0.5513
 -> w_LoRA1 = 0.5513
 -> w_LoRA2 = 0.4487
Fitness BLEU gabungan terbaik = 21.8515

=== HASIL AKHIR GA ===
alpha terbaik    : 0.5513
w_LoRA1 (adapter1): 0.5513
w_LoRA2 (adapter2): 0.4487
fitness (BLEU mix): 21.8515
Total waktu GA     : 2922.63 detik

=== MEMORI GPU ===
Memori awal  : 6641.43 MB
Memori akhir : 6641.43 MB
Peak usage   : 12018.80 MB

=== MEMORI RAM ===
RAM awal  : 4860.39 MB
RAM akhir : 4876.03 MB
Δ RAM     : 15.64 MB


In [ ]:
import psutil, os, time

process = psutil.Process(os.getpid())

if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.empty_cache()

start_gpu_mem = torch.cuda.memory_allocated()
start_ram = process.memory_info().rss
start_time = time.perf_counter()

best_alpha, best_fitness, ga_history = run_ga_for_lora_soup(
    eval_en_dataset=test_en_samples,
    eval_id_dataset=test_id_samples,
    pop_size=6,
    n_generations=4,
    elite_frac=1.0,
)

end_time = time.perf_counter()
end_ram = process.memory_info().rss
end_gpu_mem = torch.cuda.memory_allocated()
peak_gpu_mem = torch.cuda.max_memory_allocated()

elapsed = end_time - start_time

print("\n=== HASIL AKHIR GA ===")
print(f"alpha terbaik    : {best_alpha:.4f}")
print(f"w_LoRA1 (adapter1): {best_alpha:.4f}")
print(f"w_LoRA2 (adapter2): {1.0 - best_alpha:.4f}")
print(f"fitness (BLEU mix): {best_fitness:.4f}")
print(f"Total waktu GA     : {elapsed:.2f} detik")

print("\n=== MEMORI GPU ===")
print(f"Memori awal  : {start_gpu_mem / (1024**2):.2f} MB")
print(f"Memori akhir : {end_gpu_mem / (1024**2):.2f} MB")
print(f"Peak usage   : {peak_gpu_mem / (1024**2):.2f} MB")

print("\n=== MEMORI RAM ===")
print(f"RAM awal  : {start_ram / (1024**2):.2f} MB")
print(f"RAM akhir : {end_ram / (1024**2):.2f} MB")
print(f"Δ RAM     : {(end_ram - start_ram) / (1024**2):.2f} MB")


=== Generasi 1/4
  Individu 1/6: alpha=0.6965
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.84s/it]


[alpha=0.6965] BLEU_EN=12.4092, BLEU_ID=7.2880, Fitness=9.8486
  Individu 2/6: alpha=0.2861
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.82s/it]


[alpha=0.2861] BLEU_EN=19.0632, BLEU_ID=4.7788, Fitness=11.9210
  Individu 3/6: alpha=0.2269
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

[alpha=0.7195] BLEU_EN=13.8903, BLEU_ID=10.9171, Fitness=12.4037
  Individu 5/6: alpha=0.2861
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.82s/it]


[alpha=0.2861] BLEU_EN=19.0632, BLEU_ID=4.7788, Fitness=11.9210
  Individu 6/6: alpha=0.6965
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.79s/it]


[alpha=0.6965] BLEU_EN=12.4092, BLEU_ID=7.2880, Fitness=9.8486
 >> Best generasi 4: alpha=0.5513, fitness=21.8515
 >> Global best sejauh ini: alpha=0.5513, fitness=21.8515

=== GA selesai
Bobot terbaik (alpha) = 0.5513
 -> w_LoRA1 = 0.5513
 -> w_LoRA2 = 0.4487
Fitness BLEU gabungan terbaik = 21.8515

=== HASIL AKHIR GA ===
alpha terbaik    : 0.5513
w_LoRA1 (adapter1): 0.5513
w_LoRA2 (adapter2): 0.4487
fitness (BLEU mix): 21.8515
Total waktu GA     : 1462.50 detik

=== MEMORI GPU ===
Memori awal  : 6641.43 MB
Memori akhir : 6641.43 MB
Peak usage   : 12018.80 MB

=== MEMORI RAM ===
RAM awal  : 4856.73 MB
RAM akhir : 4860.29 MB
Δ RAM     : 3.55 MB


### Building the adapter

In [ ]:
# Configuration
BASE_MODEL = "unsloth/Llama-3.2-1B-Instruct-unsloth-bnb-4bit"

LORA1_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-id"
LORA2_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-en"

LORA_SOUP_DIR = "/content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_1"

CAT_WEIGHTS = [0.5513, 0.4487]

LOAD_IN_4BIT = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

Device: cuda


In [ ]:
def show_adapter_cfg(path):
    with open(os.path.join(path, "adapter_config.json"), "r") as f:
        cfg = json.load(f)
    print("===", path, "===")
    print("r           :", cfg["r"])
    print("lora_alpha  :", cfg.get("lora_alpha"))
    print("target_modules:", cfg.get("target_modules"))

show_adapter_cfg(LORA1_DIR)
show_adapter_cfg(LORA2_DIR)

=== /content/drive/MyDrive/College/KK/Final Project/model/lora-id ===
r           : 16
lora_alpha  : 16
target_modules: ['k_proj', 'o_proj', 'up_proj', 'down_proj', 'v_proj', 'gate_proj', 'q_proj']
=== /content/drive/MyDrive/College/KK/Final Project/model/lora-en ===
r           : 16
lora_alpha  : 16
target_modules: ['q_proj', 'o_proj', 'down_proj', 'v_proj', 'up_proj', 'k_proj', 'gate_proj']


In [ ]:
import os, json, shutil
from typing import List, Optional

import torch
from safetensors.torch import load_file, save_file

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def build_lora_soup(
    lora_paths: List[str],
    weights: Optional[List[float]],
    out_path: str,
    copy_tokenizer: bool = True,
):
    """
    Menggabungkan beberapa LoRA menjadi 1 adapter LoRA-Soups
    dengan cara *weighted average* (model soup) atas seluruh
    bobot float di state_dict.

    lora_paths : list path adapter LoRA sumber
    weights    : bobot soup, akan dinormalisasi; jika None -> uniform
    out_path   : folder output adapter baru
    """
    assert len(lora_paths) >= 2, "Minimal 2 LoRA untuk LoRA-Soups"

    # jika weights tidak diberikan -> uniform
    if weights is None:
        w = torch.ones(len(lora_paths), dtype=torch.float32)
    else:
        assert len(weights) == len(lora_paths), "Panjang weights harus = jumlah LoRA"
        w = torch.tensor(weights, dtype=torch.float32)

    # normalisasi bobot
    w = w / w.sum()

    os.makedirs(out_path, exist_ok=True)

    # 1. baca config dari adapter pertama
    cfg_path0 = os.path.join(lora_paths[0], "adapter_config.json")
    with open(cfg_path0, "r") as f:
        cfg0 = json.load(f)

    base_r      = cfg0["r"]
    base_alpha  = cfg0.get("lora_alpha", None)
    base_target = set(cfg0.get("target_modules", []))

    # cek konsistensi adapter lain
    for p in lora_paths[1:]:
        with open(os.path.join(p, "adapter_config.json"), "r") as f:
            cfgi = json.load(f)
        assert cfgi["r"] == base_r, f"Rank LoRA beda di {p}"
        if "lora_alpha" in cfgi and base_alpha is not None:
            assert cfgi["lora_alpha"] == base_alpha, f"lora_alpha beda di {p}"
        if "target_modules" in cfgi and base_target:
            assert set(cfgi["target_modules"]) == base_target, f"target_modules beda di {p}"

    # 2. config baru untuk SOUP:
    #    rank r & lora_alpha TIDAK berubah, hanya bobot yang di-average
    cfg_soup = cfg0.copy()
    if "target_modules" in cfg_soup:
        cfg_soup["target_modules"] = sorted(cfg_soup["target_modules"])

    with open(os.path.join(out_path, "adapter_config.json"), "w") as f:
        json.dump(cfg_soup, f, indent=2)

    # 3. load semua state dict LoRA
    state_dicts = [load_file(os.path.join(p, "adapter_model.safetensors")) for p in lora_paths]

    # opsional: pastikan key sama semua
    keys0 = set(state_dicts[0].keys())
    for i, sd in enumerate(state_dicts[1:], start=1):
        ki = set(sd.keys())
        assert keys0 == ki, f"Kumpulan key state_dict LoRA ke-{i} beda; cek adapter: {lora_paths[i]}"

    # 4. build state dict baru dengan weighted average
    new_state = {}
    float_dtypes = {
        torch.float16,
        torch.bfloat16,
        torch.float32,
        torch.float64,
    }

    for key in state_dicts[0].keys():
        tensor0 = state_dicts[0][key]

        # Kalau tensor floating -> soup (weighted average)
        if isinstance(tensor0, torch.Tensor) and tensor0.dtype in float_dtypes:
            acc = torch.zeros_like(tensor0)
            for wi, sd in zip(w, state_dicts):
                acc = acc + wi * sd[key]
            new_state[key] = acc
        else:
            # Non-float (misal buffer int, dsb) -> ambil dari adapter pertama saja
            new_state[key] = tensor0

    # 5. simpan
    save_file(new_state, os.path.join(out_path, "adapter_model.safetensors"))

    # 6. copy tokenizer & template dari adapter pertama (opsional)
    if copy_tokenizer:
        extra_files = [
            "tokenizer.json",
            "tokenizer_config.json",
            "special_tokens_map.json",
            "chat_template.jinja",
            "README.md",
        ]
        for fname in extra_files:
            src = os.path.join(lora_paths[0], fname)
            if os.path.exists(src):
                shutil.copy(src, os.path.join(out_path, fname))

    print(f"[OK] LoRA-Soups disimpan di: {out_path}")

In [ ]:
build_lora_soup(
    lora_paths=[LORA1_DIR, LORA2_DIR],
    weights=CAT_WEIGHTS,
    out_path=LORA_SOUP_DIR,
    copy_tokenizer=True,
)

[OK] LoRA-Soups disimpan di: /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_1


### Evaluation

In [85]:
from sacrebleu.metrics import BLEU
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import random
import transformers
import os
from unsloth import FastLanguageModel
from rouge_score import rouge_scorer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    transformers.set_seed(seed)

# System prompt per bahasa
SYSTEM_PROMPT_ID = (
    "Anda adalah asisten AI yang membantu pengguna dalam bahasa Indonesia. "
)

SYSTEM_PROMPT_EN = (
    "You are a helpful AI assistant that communicates in English. "
)

MAX_SEQ_LEN      = 1024   # sama dengan di FastLanguageModel.from_pretrained
MAX_NEW_TOKENS   = 128
MAX_INPUT_LENGTH = MAX_SEQ_LEN - MAX_NEW_TOKENS  # supaya total <= 1024

def make_collate_fn(tokenizer, system_prompt, schema):
    def collate_fn(batch):
        input_id_list = []
        references = []

        for item in batch:
            if schema == "id":
                input_text = item["input"]      # ID: input/output
                reference  = item["output"]
            elif schema == "en":
                input_text = item["question"]   # EN: question/response
                reference  = item["response"]
            else:
                raise ValueError(f"Unknown schema: {schema}")

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": input_text},
            ]

            # 1) Tokenisasi per-sample TANPA padding batch
            ids = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
                truncation=True,
                max_length=MAX_INPUT_LENGTH,
            )[0]  # shape: (seq_len,)

            input_id_list.append(ids)
            references.append(reference)

        # 2) Hitung panjang maksimum di batch
        max_len = max(t.shape[0] for t in input_id_list)
        pad_id  = tokenizer.pad_token_id
        batch_size = len(input_id_list)

        # 3) Siapkan tensor batched dengan LEFT PADDING manual
        input_ids = torch.full(
            (batch_size, max_len),
            pad_id,
            dtype=torch.long,
        )
        attention_mask = torch.zeros(
            (batch_size, max_len),
            dtype=torch.long,
        )

        for i, ids in enumerate(input_id_list):
            seq_len = ids.shape[0]
            # left padding: isi dari kanan
            input_ids[i, -seq_len:] = ids
            attention_mask[i, -seq_len:] = 1

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "references": references,
        }

    return collate_fn

def evaluate_model(model, tokenizer, eval_dataset, system_prompt,
                   batch_size=64, max_allowed_length=24, schema="en"):
    model.eval()
    all_predictions = []
    all_references = []        # untuk BLEU (nested list)
    all_references_text = []   # untuk ROUGE (list string)

    collate_fn = make_collate_fn(tokenizer, system_prompt, schema)

    dataloader = DataLoader(
        eval_dataset,
        num_workers=8,              # 0 dulu biar error kelihatan jelas
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=False,
        pin_memory=True,
    )

    bleu = BLEU()
    pbar = tqdm(dataloader)

    for batch in pbar:
        input_ids = batch["input_ids"].to(DEVICE)          # shape: (B, L)
        attention_mask = batch["attention_mask"].to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_allowed_length,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=False,
                use_cache=True,
            )

        # semua sequence punya panjang awal yang sama: L
        seq_len = input_ids.shape[1]

        predictions = []
        for i, output in enumerate(outputs):
            # ambil token hasil generate saja (setelah input penuh)
            generated_tokens = output[seq_len:]
            decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True)
            predictions.append(decoded)

        all_predictions.extend(predictions)
        all_references.extend([[ref] for ref in batch["references"]])
        all_references_text.extend(batch["references"])

    # ====== BLEU ======
    final_bleu = bleu.corpus_score(all_predictions, all_references)

    # ====== ROUGE-1, ROUGE-2, ROUGE-L ======
    scorer = rouge_scorer.RougeScorer(
        ["rouge1", "rouge2", "rougeL"],
        use_stemmer=True,
    )

    rouge1_list, rouge2_list, rougeL_list = [], [], []

    for pred, ref in zip(all_predictions, all_references_text):
        scores = scorer.score(ref, pred)
        rouge1_list.append(scores["rouge1"].fmeasure)
        rouge2_list.append(scores["rouge2"].fmeasure)
        rougeL_list.append(scores["rougeL"].fmeasure)

    # kali 100 supaya bentuknya persen seperti BLEU
    rouge1 = float(np.mean(rouge1_list) * 100)
    rouge2 = float(np.mean(rouge2_list) * 100)
    rougeL = float(np.mean(rougeL_list) * 100)

    return {
        "bleu": final_bleu.score,
        "rouge1": rouge1,
        "rouge2": rouge2,
        "rougeL": rougeL,
        "predictions": all_predictions,
        "references": all_references,
    }


def evaluate_model_multiple_runs(
    model,
    tokenizer,
    eval_dataset,
    system_prompt,
    n_runs=3,
    batch_size=128,
    max_allowed_length=24,
    schema="en",
):
    all_results = []

    for run in range(n_runs):
        set_seed(42 + run)
        results = evaluate_model(
            model,
            tokenizer,
            eval_dataset,
            system_prompt=system_prompt,
            batch_size=batch_size,
            max_allowed_length=max_allowed_length,
            schema=schema,
        )
        all_results.append(results)

        bleu_scores   = [r["bleu"]   for r in all_results]
        rouge1_scores = [r["rouge1"] for r in all_results]
        rouge2_scores = [r["rouge2"] for r in all_results]
        rougeL_scores = [r["rougeL"] for r in all_results]

        print(
            f"Run {run + 1}/{n_runs} - "
            f"BLEU: {np.mean(bleu_scores):.4f} ± {np.std(bleu_scores):.4f} | "
            f"R1: {np.mean(rouge1_scores):.4f} | "
            f"R2: {np.mean(rouge2_scores):.4f} | "
            f"RL: {np.mean(rougeL_scores):.4f}"
        )

    bleu_scores   = [r["bleu"]   for r in all_results]
    rouge1_scores = [r["rouge1"] for r in all_results]
    rouge2_scores = [r["rouge2"] for r in all_results]
    rougeL_scores = [r["rougeL"] for r in all_results]

    return {
        "bleu_mean":   float(np.mean(bleu_scores)),
        "bleu_std":    float(np.std(bleu_scores)),
        "rouge1_mean": float(np.mean(rouge1_scores)),
        "rouge1_std":  float(np.std(rouge1_scores)),
        "rouge2_mean": float(np.mean(rouge2_scores)),
        "rouge2_std":  float(np.std(rouge2_scores)),
        "rougeL_mean": float(np.mean(rougeL_scores)),
        "rougeL_std":  float(np.std(rougeL_scores)),
    }

MODEL_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_1"
print(MODEL_DIR)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_DIR,
    max_seq_length=MAX_SEQ_LEN,
    dtype=None,
    load_in_4bit=True,
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.padding_side = "left"

model.eval()
FastLanguageModel.for_inference(model)

results_en_detail = evaluate_model(
    model,
    tokenizer,
    eval_en_clean["test"],
    system_prompt=SYSTEM_PROMPT_EN,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="en",
)

results_id_detail = evaluate_model(
    model,
    tokenizer,
    eval_id_clean["test"],
    system_prompt=SYSTEM_PROMPT_ID,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="id",
)

print("\nFinal Results (test split):")
print(
    f"English (eval_en_clean['test']): "
    f"BLEU = {results_en_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_en_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_en_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_en_detail['rougeL']:.4f}"
)
print(
    f"Indonesian (eval_id_clean['test']): "
    f"BLEU = {results_id_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_id_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_id_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_id_detail['rougeL']:.4f}"
)

/content/drive/MyDrive/College/KK/Final Project/model/lora_soup_ga_1
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 34/34 [05:22<00:00,  9.48s/it]



Final Results (test split):
English (eval_en_clean['test']): BLEU = 54.8866, ROUGE-1 = 34.3824, ROUGE-2 = 20.7656, ROUGE-L = 28.9185
Indonesian (eval_id_clean['test']): BLEU = 54.6633, ROUGE-1 = 16.6658, ROUGE-2 = 5.3391, ROUGE-L = 13.9115


## LoRA-Soups (CMA-ES)

### Search for the best weight

In [ ]:
N_HEAD = 10
N_MID = 30
N_TAIL = 10
N = N_HEAD + N_MID + N_TAIL

ds_id = eval_id_clean["test"]
ds_en = eval_en_clean["test"]

n_total = len(ds_id)
assert len(ds_en) == n_total, "Panjang split ID dan EN harus sama"
assert n_total >= N, "Dataset terlalu kecil untuk ambil 50 sampel"

head_indices = list(range(0, N_HEAD))

mid_start = (n_total - N_MID) // 2
mid_indices = list(range(mid_start, mid_start + N_MID))

tail_indices = list(range(n_total - N_TAIL, n_total))

indices = head_indices + mid_indices + tail_indices

test_id_samples = ds_id.select(indices)
test_en_samples = ds_en.select(indices)

display(test_id_samples)
display(test_en_samples)

Dataset({
    features: ['dataset_name', 'subset_name', 'prompt_id', 'template_name', 'dataset_key', 'input', 'output'],
    num_rows: 50
})

Dataset({
    features: ['id', 'system_prompt', 'question', 'response'],
    num_rows: 50
})

In [ ]:
from safetensors.torch import load_file, save_file
import shutil

LORA1_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-id"
LORA2_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-en"

LORA_SOUP_TMP_DIR = "/content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp"

In [ ]:
FLOAT_DTYPES = {
    torch.float16,
    torch.bfloat16,
    torch.float32,
    torch.float64,
}


def load_two_lora_state_dicts(lora_dir_1, lora_dir_2):
    """
    Load state_dict dari 2 adapter LoRA (safetensors).
    Pastikan kedua adapter dilatih dari base dan config yang sama.
    """
    sd1 = load_file(os.path.join(lora_dir_1, "adapter_model.safetensors"))
    sd2 = load_file(os.path.join(lora_dir_2, "adapter_model.safetensors"))

    keys1 = set(sd1.keys())
    keys2 = set(sd2.keys())
    if keys1 != keys2:
        raise ValueError("Key di adapter LoRA 1 dan 2 tidak sama. "
                         "Pastikan keduanya dilatih dari base + config yang sama.")

    return sd1, sd2


def prepare_soup_dir(template_dir, soup_dir):
    """
    Membuat folder soup sekali saja dengan cara copy seluruh isi template_dir
    (misal dari LORA1_DIR) ke soup_dir.
    Nanti setiap kali GA update bobot, kita hanya overwrite adapter_model.safetensors di soup_dir.
    """
    if os.path.exists(soup_dir):
        shutil.rmtree(soup_dir)
    shutil.copytree(template_dir, soup_dir)
    print(f"[INFO] Folder soup dibuat dari template: {template_dir} -> {soup_dir}")


def write_soup_state_dict(sd1, sd2, alpha, out_dir):
    """
    Membuat LoRA-Soup dari 2 adapter:
        W_soup = alpha * W1 + (1 - alpha) * W2

    alpha di [0, 1]. Hasilnya disimpan ke adapter_model.safetensors di out_dir.
    """
    alpha = float(alpha)
    if not (0.0 <= alpha <= 1.0):
        raise ValueError("alpha harus di antara 0 dan 1")

    w1 = alpha
    w2 = 1.0 - alpha

    new_state = {}
    for k in sd1.keys():
        t1 = sd1[k]
        t2 = sd2[k]

        if isinstance(t1, torch.Tensor) and t1.dtype in FLOAT_DTYPES:
            new_state[k] = w1 * t1 + w2 * t2
        else:
            new_state[k] = t1

    out_path = os.path.join(out_dir, "adapter_model.safetensors")
    save_file(new_state, out_path)
    print(f"[DEBUG] Soup weights disimpan ke {out_path}")

In [ ]:
# Sanity Check LoRA
lora_sd1, lora_sd2 = load_two_lora_state_dicts(LORA1_DIR, LORA2_DIR)
prepare_soup_dir(LORA1_DIR, LORA_SOUP_TMP_DIR)

[INFO] Folder soup dibuat dari template: /content/drive/MyDrive/College/KK/Final Project/model/lora-id -> /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp


In [ ]:
from sacrebleu.metrics import BLEU
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import random
import transformers
import os
from unsloth import FastLanguageModel
from rouge_score import rouge_scorer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    transformers.set_seed(seed)

SYSTEM_PROMPT_ID = (
    "Anda adalah asisten AI yang membantu pengguna dalam bahasa Indonesia. "
)

SYSTEM_PROMPT_EN = (
    "You are a helpful AI assistant that communicates in English. "
)

MAX_SEQ_LEN      = 1024
MAX_NEW_TOKENS   = 128
MAX_INPUT_LENGTH = MAX_SEQ_LEN - MAX_NEW_TOKENS

def make_collate_fn(tokenizer, system_prompt, schema):
    def collate_fn(batch):
        input_id_list = []
        references = []

        for item in batch:
            if schema == "id":
                input_text = item["input"]      # ID: input/output
                reference  = item["output"]
            elif schema == "en":
                input_text = item["question"]   # EN: question/response
                reference  = item["response"]
            else:
                raise ValueError(f"Unknown schema: {schema}")

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": input_text},
            ]

            # 1) Tokenisasi per-sample TANPA padding batch
            ids = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
                truncation=True,
                max_length=MAX_INPUT_LENGTH,
            )[0]  # shape: (seq_len,)

            input_id_list.append(ids)
            references.append(reference)

        # 2) Hitung panjang maksimum di batch
        max_len = max(t.shape[0] for t in input_id_list)
        pad_id  = tokenizer.pad_token_id
        batch_size = len(input_id_list)

        # 3) Siapkan tensor batched dengan LEFT PADDING manual
        input_ids = torch.full(
            (batch_size, max_len),
            pad_id,
            dtype=torch.long,
        )
        attention_mask = torch.zeros(
            (batch_size, max_len),
            dtype=torch.long,
        )

        for i, ids in enumerate(input_id_list):
            seq_len = ids.shape[0]
            # left padding: isi dari kanan
            input_ids[i, -seq_len:] = ids
            attention_mask[i, -seq_len:] = 1

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "references": references,
        }

    return collate_fn

def evaluate_model(model, tokenizer, eval_dataset, system_prompt,
                   batch_size=64, max_allowed_length=24, schema="en"):
    model.eval()
    all_predictions = []
    all_references = []        # untuk BLEU (nested list)
    all_references_text = []   # untuk ROUGE (list string)

    collate_fn = make_collate_fn(tokenizer, system_prompt, schema)

    dataloader = DataLoader(
        eval_dataset,
        num_workers=8,              # 0 dulu biar error kelihatan jelas
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=False,
        pin_memory=True,
    )

    bleu = BLEU()
    pbar = tqdm(dataloader)

    for batch in pbar:
        input_ids = batch["input_ids"].to(DEVICE)          # shape: (B, L)
        attention_mask = batch["attention_mask"].to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_allowed_length,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=False,
                use_cache=True,
            )

        # semua sequence punya panjang awal yang sama: L
        seq_len = input_ids.shape[1]

        predictions = []
        for i, output in enumerate(outputs):
            # ambil token hasil generate saja (setelah input penuh)
            generated_tokens = output[seq_len:]
            decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True)
            predictions.append(decoded)

        all_predictions.extend(predictions)
        all_references.extend([[ref] for ref in batch["references"]])
        all_references_text.extend(batch["references"])

    # ====== BLEU ======
    final_bleu = bleu.corpus_score(all_predictions, all_references)

    # ====== ROUGE-1, ROUGE-2, ROUGE-L ======
    scorer = rouge_scorer.RougeScorer(
        ["rouge1", "rouge2", "rougeL"],
        use_stemmer=True,
    )

    rouge1_list, rouge2_list, rougeL_list = [], [], []

    for pred, ref in zip(all_predictions, all_references_text):
        scores = scorer.score(ref, pred)
        rouge1_list.append(scores["rouge1"].fmeasure)
        rouge2_list.append(scores["rouge2"].fmeasure)
        rougeL_list.append(scores["rougeL"].fmeasure)

    # kali 100 supaya bentuknya persen seperti BLEU
    rouge1 = float(np.mean(rouge1_list) * 100)
    rouge2 = float(np.mean(rouge2_list) * 100)
    rougeL = float(np.mean(rougeL_list) * 100)

    return {
        "bleu": final_bleu.score,
        "rouge1": rouge1,
        "rouge2": rouge2,
        "rougeL": rougeL,
        "predictions": all_predictions,
        "references": all_references,
    }


def evaluate_model_multiple_runs(
    model,
    tokenizer,
    eval_dataset,
    system_prompt,
    n_runs=3,
    batch_size=128,
    max_allowed_length=24,
    schema="en",
):
    all_results = []

    for run in range(n_runs):
        set_seed(42 + run)
        results = evaluate_model(
            model,
            tokenizer,
            eval_dataset,
            system_prompt=system_prompt,
            batch_size=batch_size,
            max_allowed_length=max_allowed_length,
            schema=schema,
        )
        all_results.append(results)

        bleu_scores   = [r["bleu"]   for r in all_results]
        rouge1_scores = [r["rouge1"] for r in all_results]
        rouge2_scores = [r["rouge2"] for r in all_results]
        rougeL_scores = [r["rougeL"] for r in all_results]

        print(
            f"Run {run + 1}/{n_runs} - "
            f"BLEU: {np.mean(bleu_scores):.4f} ± {np.std(bleu_scores):.4f} | "
            f"R1: {np.mean(rouge1_scores):.4f} | "
            f"R2: {np.mean(rouge2_scores):.4f} | "
            f"RL: {np.mean(rougeL_scores):.4f}"
        )

    bleu_scores   = [r["bleu"]   for r in all_results]
    rouge1_scores = [r["rouge1"] for r in all_results]
    rouge2_scores = [r["rouge2"] for r in all_results]
    rougeL_scores = [r["rougeL"] for r in all_results]

    return {
        "bleu_mean":   float(np.mean(bleu_scores)),
        "bleu_std":    float(np.std(bleu_scores)),
        "rouge1_mean": float(np.mean(rouge1_scores)),
        "rouge1_std":  float(np.std(rouge1_scores)),
        "rouge2_mean": float(np.mean(rouge2_scores)),
        "rouge2_std":  float(np.std(rouge2_scores)),
        "rougeL_mean": float(np.mean(rougeL_scores)),
        "rougeL_std":  float(np.std(rougeL_scores)),
    }

def evaluate_alpha(alpha, eval_en_dataset, eval_id_dataset, batch_size=75, verbose=True, weight_id=0.5):
    """
    Evaluasi 1 nilai alpha:
        w1 = alpha, w2 = 1 - alpha
    Fitness = (1 - weight_id)*BLEU_EN + weight_id*BLEU_ID
    """

    # 1. Tulis LoRA-Soup ke LORA_SOUP_TMP_DIR
    write_soup_state_dict(lora_sd1, lora_sd2, alpha, LORA_SOUP_TMP_DIR)

    # 2. Bersihkan GPU sebelum load model baru
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # 3. Load model + tokenizer dari adapter soup
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=LORA_SOUP_TMP_DIR,
        max_seq_length=MAX_SEQ_LEN,
        dtype=None,
        load_in_4bit=True,
    )

    # Set pad token dan padding side sesuai evaluasi sebelumnya
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id
    tokenizer.padding_side = "left"

    model.to(DEVICE)
    model.eval()
    FastLanguageModel.for_inference(model)

    results_en = evaluate_model(
        model,
        tokenizer,
        eval_en_dataset,
        system_prompt=SYSTEM_PROMPT_EN,
        batch_size=batch_size,
        max_allowed_length=MAX_NEW_TOKENS,
        schema="en",
    )

    results_id = evaluate_model(
        model,
        tokenizer,
        eval_id_dataset,
        system_prompt=SYSTEM_PROMPT_ID,
        batch_size=batch_size,
        max_allowed_length=MAX_NEW_TOKENS,
        schema="id",
    )

    bleu_en = results_en["bleu"]
    bleu_id = results_id["bleu"]

    fitness = (1.0 - weight_id) * bleu_en + weight_id * bleu_id

    if verbose:
        print(
            f"[alpha={alpha:.4f}] "
            f"BLEU_EN={bleu_en:.4f}, BLEU_ID={bleu_id:.4f}, "
            f"Fitness={fitness:.4f}"
        )

    del model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return fitness, bleu_en, bleu_id

In [ ]:
# Quick Test
fit_05, bleu_en_05, bleu_id_05 = evaluate_alpha(
    alpha=0.5,
    eval_en_dataset=test_en_samples,
    eval_id_dataset=test_id_samples,
)
print("Fitness alpha=0.5:", fit_05)

[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.74s/it]


[alpha=0.5000] BLEU_EN=27.9623, BLEU_ID=7.7550, Fitness=17.8587
Fitness alpha=0.5: 17.858654659790623


In [ ]:
!pip install -q cma

In [ ]:
import numpy as np
import cma


def cma_objective_alpha_2d(
    z_vec,
    eval_en_dataset,
    eval_id_dataset,
    weight_id=0.5,
    batch_size=75,
    eval_verbose=False,
):
    """
    Objective untuk CMA-ES di dimensi 2.

    z_vec: array-like panjang 2 -> [z1, z2]
    Kita map z1 -> alpha ∈ (0, 1) dengan sigmoid.
    z2 diabaikan (dummy dimension) hanya supaya n >= 2.
    """
    z_vec = np.asarray(z_vec, dtype=float)
    z1 = float(z_vec[0])

    # Map ke alpha ∈ (0,1)
    alpha = 1.0 / (1.0 + np.exp(-z1))

    # Evaluasi LoRA-Soup seperti biasa
    fitness, bleu_en, bleu_id = evaluate_alpha(
        alpha=alpha,
        eval_en_dataset=eval_en_dataset,
        eval_id_dataset=eval_id_dataset,
        batch_size=batch_size,
        verbose=eval_verbose,
        weight_id=weight_id,
    )

    # CMA-ES always MINIMIZE -> kembalikan -fitness (biar maximize fitness)
    return -fitness

def run_cmaes_for_lora_soup(
    eval_en_dataset,
    eval_id_dataset,
    pop_size=8,
    max_iter=20,
    sigma0=1.0,
    z0=(0.0, 0.0),
    weight_id=0.5,
    batch_size=75,
    seed=42,
    eval_verbose=False,
):
    """
    Jalankan CMA-ES (dimensi 2) untuk mencari alpha terbaik LoRA-Soup.

    - Kita optimasi z ∈ R^2.
    - alpha = sigmoid(z1) ∈ (0,1).
    - w_LoRA1 = alpha
      w_LoRA2 = 1 - alpha

    Argumen mirip versi GA/CMA 1D sebelumnya.
    """

    # Set random seed global (sesuai utilmu)
    set_seed(seed)

    # Opsi CMA-ES
    options = {
        # batas untuk z1/z2; setelah sigmoid, alpha hampir [0,1]
        "bounds": [-8.0, 8.0],
        "popsize": pop_size,
        "maxiter": max_iter,
        "verb_disp": 1,
        "seed": seed,
    }

    # Bungkus objective supaya dataset & parameter lain "tertutup"
    def _objective(z_vec):
        return cma_objective_alpha_2d(
            z_vec,
            eval_en_dataset=eval_en_dataset,
            eval_id_dataset=eval_id_dataset,
            weight_id=weight_id,
            batch_size=batch_size,
            eval_verbose=eval_verbose,
        )

    # Inisialisasi CMA-ES di dimensi 2
    es = cma.CMAEvolutionStrategy(list(z0), sigma0, options)

    history = []  # untuk simpan log tiap evaluasi

    eval_counter = 0
    while not es.stop():
        # 1) Minta populasi kandidat z
        solutions = es.ask()  # list of length pop_size, tiap item array shape (2,)

        # 2) Hitung fitness
        fitnesses = []
        for z_vec in solutions:
            f = _objective(z_vec)  # = -fitness_asli
            fitnesses.append(f)

            z1 = float(z_vec[0])
            z2 = float(z_vec[1])
            alpha = 1.0 / (1.0 + np.exp(-z1))

            history.append(
                {
                    "eval": eval_counter,
                    "z1": z1,
                    "z2": z2,
                    "alpha": alpha,
                    "fitness_neg": float(f),   # yang dipakai CMA-ES
                    "fitness_pos": float(-f),  # fitness asli (BLEU mix)
                }
            )
            eval_counter += 1

        # 3) Update CMA-ES dengan hasil evaluasi
        es.tell(solutions, fitnesses)
        es.disp()  # print status singkat

    # Ambil hasil terbaik dari CMA-ES
    result = es.result

    z1_best = float(result.xbest[0])
    best_alpha = 1.0 / (1.0 + np.exp(-z1_best))
    best_fitness = float(-result.fbest)

    print("\n=== HASIL AKHIR CMA-ES (2D) ===")
    print(f"alpha terbaik     : {best_alpha:.4f}")
    print(f"w_LoRA1 (adapter1): {best_alpha:.4f}")
    print(f"w_LoRA2 (adapter2): {1.0 - best_alpha:.4f}")
    print(f"fitness (BLEU mix): {best_fitness:.4f}")

    return best_alpha, best_fitness, history

In [ ]:
import psutil, os, time

process = psutil.Process(os.getpid())

if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.empty_cache()

start_gpu_mem = torch.cuda.memory_allocated()
start_ram = process.memory_info().rss
start_time = time.perf_counter()

best_alpha, best_fitness, cma_history = run_cmaes_for_lora_soup(
    eval_en_dataset=test_en_samples,
    eval_id_dataset=test_id_samples,
    pop_size=8,
    max_iter=6,
    sigma0=0.25,          # bisa kamu tuning
    z0=(0.0, 0.0),       # alpha awal ~ 0.5
    weight_id=0.5,
    batch_size=80,
    seed=42,
    eval_verbose=True,
)


end_time = time.perf_counter()
end_ram = process.memory_info().rss
end_gpu_mem = torch.cuda.memory_allocated()
peak_gpu_mem = torch.cuda.max_memory_allocated()

elapsed = end_time - start_time

print("\n=== HASIL AKHIR CMA-ES ===")
print(f"alpha terbaik    : {best_alpha:.4f}")
print(f"w_LoRA1 (adapter1): {best_alpha:.4f}")
print(f"w_LoRA2 (adapter2): {1.0 - best_alpha:.4f}")
print(f"fitness (BLEU mix): {best_fitness:.4f}")
print(f"Total waktu CMA-ES     : {elapsed:.2f} detik")

print("\n=== MEMORI GPU ===")
print(f"Memori awal  : {start_gpu_mem / (1024**2):.2f} MB")
print(f"Memori akhir : {end_gpu_mem / (1024**2):.2f} MB")
print(f"Peak usage   : {peak_gpu_mem / (1024**2):.2f} MB")

print("\n=== MEMORI RAM ===")
print(f"RAM awal  : {start_ram / (1024**2):.2f} MB")
print(f"RAM akhir : {end_ram / (1024**2):.2f} MB")
print(f"Δ RAM     : {(end_ram - start_ram) / (1024**2):.2f} MB")

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 2 (seed=42, Tue Dec  2 10:49:57 2025)
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.75s/it]


[alpha=0.5310] BLEU_EN=34.6064, BLEU_ID=13.4851, Fitness=24.0458
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.63s/it]


[alpha=0.5404] BLEU_EN=27.9623, BLEU_ID=13.4851, Fitness=20.7237
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

[alpha=0.5477] BLEU_EN=27.9623, BLEU_ID=13.4851, Fitness=20.7237
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.70s/it]


[alpha=0.5700] BLEU_EN=34.6066, BLEU_ID=13.4851, Fitness=24.0459
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.77s/it]


[alpha=0.5654] BLEU_EN=34.6066, BLEU_ID=13.4851, Fitness=24.0459
    6     48 -2.404587871064248e+01 2.0e+00 1.74e-01  8e-02  2e-01 48:07.6

=== HASIL AKHIR CMA-ES (2D) ===
alpha terbaik     : 0.5644
w_LoRA1 (adapter1): 0.5644
w_LoRA2 (adapter2): 0.4356
fitness (BLEU mix): 24.0459

=== HASIL AKHIR GA ===
alpha terbaik    : 0.5644
w_LoRA1 (adapter1): 0.5644
w_LoRA2 (adapter2): 0.4356
fitness (BLEU mix): 24.0459
Total waktu GA     : 2887.66 detik

=== MEMORI GPU ===
Memori awal  : 3862.24 MB
Memori akhir : 3862.24 MB
Peak usage   : 9239.61 MB

=== MEMORI RAM ===
RAM awal  : 4762.96 MB
RAM akhir : 4783.15 MB
Δ RAM     : 20.19 MB


In [ ]:
import psutil, os, time

process = psutil.Process(os.getpid())

if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.empty_cache()

start_gpu_mem = torch.cuda.memory_allocated()
start_ram = process.memory_info().rss
start_time = time.perf_counter()

best_alpha_A, best_fit_A, hist_A = run_cmaes_for_lora_soup(
    eval_en_dataset=test_en_samples,
    eval_id_dataset=test_id_samples,
    pop_size=6,
    max_iter=4,
    sigma0=1.0,
    z0=(0.0, 0.0),
    weight_id=0.5,
    batch_size=80,
    seed=42,
    eval_verbose=True,
)

end_time = time.perf_counter()
end_ram = process.memory_info().rss
end_gpu_mem = torch.cuda.memory_allocated()
peak_gpu_mem = torch.cuda.max_memory_allocated()

elapsed = end_time - start_time

print("\n=== HASIL AKHIR CMA-ES ===")
print(f"alpha terbaik    : {best_alpha:.4f}")
print(f"w_LoRA1 (adapter1): {best_alpha:.4f}")
print(f"w_LoRA2 (adapter2): {1.0 - best_alpha:.4f}")
print(f"fitness (BLEU mix): {best_fitness:.4f}")
print(f"Total waktu CMA-ES     : {elapsed:.2f} detik")

print("\n=== MEMORI GPU ===")
print(f"Memori awal  : {start_gpu_mem / (1024**2):.2f} MB")
print(f"Memori akhir : {end_gpu_mem / (1024**2):.2f} MB")
print(f"Peak usage   : {peak_gpu_mem / (1024**2):.2f} MB")

print("\n=== MEMORI RAM ===")
print(f"RAM awal  : {start_ram / (1024**2):.2f} MB")
print(f"RAM akhir : {end_ram / (1024**2):.2f} MB")
print(f"Δ RAM     : {(end_ram - start_ram) / (1024**2):.2f} MB")

(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=42, Tue Dec  2 11:55:39 2025)
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.80s/it]


[alpha=0.6217] BLEU_EN=26.0271, BLEU_ID=6.4913, Fitness=16.2592
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.73s/it]


[alpha=0.6565] BLEU_EN=14.7511, BLEU_ID=7.2880, Fitness=11.0195
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

[alpha=0.5252] BLEU_EN=27.9623, BLEU_ID=13.4851, Fitness=20.7237
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.72s/it]


[alpha=0.7136] BLEU_EN=12.2346, BLEU_ID=10.9171, Fitness=11.5759
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.71s/it]


[alpha=0.4294] BLEU_EN=27.8131, BLEU_ID=7.7550, Fitness=17.7841
    4     24 -2.072369034328372e+01 1.6e+00 6.52e-01  5e-01  6e-01 24:02.8

=== HASIL AKHIR CMA-ES (2D) ===
alpha terbaik     : 0.4417
w_LoRA1 (adapter1): 0.4417
w_LoRA2 (adapter2): 0.5583
fitness (BLEU mix): 21.5345

=== HASIL AKHIR CMA-ES ===
alpha terbaik    : 0.5644
w_LoRA1 (adapter1): 0.5644
w_LoRA2 (adapter2): 0.4356
fitness (BLEU mix): 24.0459
Total waktu CMA-ES     : 1442.83 detik

=== MEMORI GPU ===
Memori awal  : 3887.55 MB
Memori akhir : 6641.43 MB
Peak usage   : 12018.80 MB

=== MEMORI RAM ===
RAM awal  : 4785.11 MB
RAM akhir : 4808.31 MB
Δ RAM     : 23.20 MB


In [ ]:
import psutil, os, time

process = psutil.Process(os.getpid())

if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.empty_cache()

start_gpu_mem = torch.cuda.memory_allocated()
start_ram = process.memory_info().rss
start_time = time.perf_counter()

best_alpha_C, best_fit_C, hist_C = run_cmaes_for_lora_soup(
    eval_en_dataset=test_en_samples,
    eval_id_dataset=test_id_samples,
    pop_size=12,
    max_iter=6,
    sigma0=1.5,
    z0=(0.0, 0.0),
    weight_id=0.5,
    batch_size=80,
    seed=44,
    eval_verbose=False,
)

end_time = time.perf_counter()
end_ram = process.memory_info().rss
end_gpu_mem = torch.cuda.memory_allocated()
peak_gpu_mem = torch.cuda.max_memory_allocated()

elapsed = end_time - start_time

print("\n=== HASIL AKHIR CMA-ES ===")
print(f"alpha terbaik    : {best_alpha:.4f}")
print(f"w_LoRA1 (adapter1): {best_alpha:.4f}")
print(f"w_LoRA2 (adapter2): {1.0 - best_alpha:.4f}")
print(f"fitness (BLEU mix): {best_fitness:.4f}")
print(f"Total waktu CMA-ES     : {elapsed:.2f} detik")

print("\n=== MEMORI GPU ===")
print(f"Memori awal  : {start_gpu_mem / (1024**2):.2f} MB")
print(f"Memori akhir : {end_gpu_mem / (1024**2):.2f} MB")
print(f"Peak usage   : {peak_gpu_mem / (1024**2):.2f} MB")

print("\n=== MEMORI RAM ===")
print(f"RAM awal  : {start_ram / (1024**2):.2f} MB")
print(f"RAM akhir : {end_ram / (1024**2):.2f} MB")
print(f"Δ RAM     : {(end_ram - start_ram) / (1024**2):.2f} MB")

(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 2 (seed=44, Tue Dec  2 12:22:02 2025)
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.72s/it]


[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.73s/it]


[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.79s/it]


[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cma_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.76s/it]


    6     72 -2.404577250834619e+01 2.8e+00 1.05e+00  3e-01  1e+00 72:31.0

=== HASIL AKHIR CMA-ES (2D) ===
alpha terbaik     : 0.5220
w_LoRA1 (adapter1): 0.5220
w_LoRA2 (adapter2): 0.4780
fitness (BLEU mix): 36.2841

=== HASIL AKHIR CMA-ES ===
alpha terbaik    : 0.5644
w_LoRA1 (adapter1): 0.5644
w_LoRA2 (adapter2): 0.4356
fitness (BLEU mix): 24.0459
Total waktu CMA-ES     : 4351.01 detik

=== MEMORI GPU ===
Memori awal  : 6641.43 MB
Memori akhir : 6641.43 MB
Peak usage   : 12018.80 MB

=== MEMORI RAM ===
RAM awal  : 4808.31 MB
RAM akhir : 4840.33 MB
Δ RAM     : 32.02 MB


### Building the adapter

In [29]:
# Configuration
BASE_MODEL = "unsloth/Llama-3.2-1B-Instruct-unsloth-bnb-4bit"

LORA1_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-id"
LORA2_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-en"

LORA_SOUP_DIR = "/content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cmaes_3"

CAT_WEIGHTS = [0.4417, 0.5583]

LOAD_IN_4BIT = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

Device: cuda


In [30]:
def show_adapter_cfg(path):
    with open(os.path.join(path, "adapter_config.json"), "r") as f:
        cfg = json.load(f)
    print("===", path, "===")
    print("r           :", cfg["r"])
    print("lora_alpha  :", cfg.get("lora_alpha"))
    print("target_modules:", cfg.get("target_modules"))

show_adapter_cfg(LORA1_DIR)
show_adapter_cfg(LORA2_DIR)

=== /content/drive/MyDrive/College/KK/Final Project/model/lora-id ===
r           : 16
lora_alpha  : 16
target_modules: ['k_proj', 'o_proj', 'up_proj', 'down_proj', 'v_proj', 'gate_proj', 'q_proj']
=== /content/drive/MyDrive/College/KK/Final Project/model/lora-en ===
r           : 16
lora_alpha  : 16
target_modules: ['q_proj', 'o_proj', 'down_proj', 'v_proj', 'up_proj', 'k_proj', 'gate_proj']


In [31]:
import os, json, shutil
from typing import List, Optional

import torch
from safetensors.torch import load_file, save_file

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def build_lora_soup(
    lora_paths: List[str],
    weights: Optional[List[float]],
    out_path: str,
    copy_tokenizer: bool = True,
):
    """
    Menggabungkan beberapa LoRA menjadi 1 adapter LoRA-Soups
    dengan cara *weighted average* (model soup) atas seluruh
    bobot float di state_dict.

    lora_paths : list path adapter LoRA sumber
    weights    : bobot soup, akan dinormalisasi; jika None -> uniform
    out_path   : folder output adapter baru
    """
    assert len(lora_paths) >= 2, "Minimal 2 LoRA untuk LoRA-Soups"

    # jika weights tidak diberikan -> uniform
    if weights is None:
        w = torch.ones(len(lora_paths), dtype=torch.float32)
    else:
        assert len(weights) == len(lora_paths), "Panjang weights harus = jumlah LoRA"
        w = torch.tensor(weights, dtype=torch.float32)

    # normalisasi bobot
    w = w / w.sum()

    os.makedirs(out_path, exist_ok=True)

    # 1. baca config dari adapter pertama
    cfg_path0 = os.path.join(lora_paths[0], "adapter_config.json")
    with open(cfg_path0, "r") as f:
        cfg0 = json.load(f)

    base_r      = cfg0["r"]
    base_alpha  = cfg0.get("lora_alpha", None)
    base_target = set(cfg0.get("target_modules", []))

    # cek konsistensi adapter lain
    for p in lora_paths[1:]:
        with open(os.path.join(p, "adapter_config.json"), "r") as f:
            cfgi = json.load(f)
        assert cfgi["r"] == base_r, f"Rank LoRA beda di {p}"
        if "lora_alpha" in cfgi and base_alpha is not None:
            assert cfgi["lora_alpha"] == base_alpha, f"lora_alpha beda di {p}"
        if "target_modules" in cfgi and base_target:
            assert set(cfgi["target_modules"]) == base_target, f"target_modules beda di {p}"

    # 2. config baru untuk SOUP:
    #    rank r & lora_alpha TIDAK berubah, hanya bobot yang di-average
    cfg_soup = cfg0.copy()
    if "target_modules" in cfg_soup:
        cfg_soup["target_modules"] = sorted(cfg_soup["target_modules"])

    with open(os.path.join(out_path, "adapter_config.json"), "w") as f:
        json.dump(cfg_soup, f, indent=2)

    # 3. load semua state dict LoRA
    state_dicts = [load_file(os.path.join(p, "adapter_model.safetensors")) for p in lora_paths]

    # opsional: pastikan key sama semua
    keys0 = set(state_dicts[0].keys())
    for i, sd in enumerate(state_dicts[1:], start=1):
        ki = set(sd.keys())
        assert keys0 == ki, f"Kumpulan key state_dict LoRA ke-{i} beda; cek adapter: {lora_paths[i]}"

    # 4. build state dict baru dengan weighted average
    new_state = {}
    float_dtypes = {
        torch.float16,
        torch.bfloat16,
        torch.float32,
        torch.float64,
    }

    for key in state_dicts[0].keys():
        tensor0 = state_dicts[0][key]

        # Kalau tensor floating -> soup (weighted average)
        if isinstance(tensor0, torch.Tensor) and tensor0.dtype in float_dtypes:
            acc = torch.zeros_like(tensor0)
            for wi, sd in zip(w, state_dicts):
                acc = acc + wi * sd[key]
            new_state[key] = acc
        else:
            # Non-float (misal buffer int, dsb) -> ambil dari adapter pertama saja
            new_state[key] = tensor0

    # 5. simpan
    save_file(new_state, os.path.join(out_path, "adapter_model.safetensors"))

    # 6. copy tokenizer & template dari adapter pertama (opsional)
    if copy_tokenizer:
        extra_files = [
            "tokenizer.json",
            "tokenizer_config.json",
            "special_tokens_map.json",
            "chat_template.jinja",
            "README.md",
        ]
        for fname in extra_files:
            src = os.path.join(lora_paths[0], fname)
            if os.path.exists(src):
                shutil.copy(src, os.path.join(out_path, fname))

    print(f"[OK] LoRA-Soups disimpan di: {out_path}")

In [32]:
build_lora_soup(
    lora_paths=[LORA1_DIR, LORA2_DIR],
    weights=CAT_WEIGHTS,
    out_path=LORA_SOUP_DIR,
    copy_tokenizer=True,
)

[OK] LoRA-Soups disimpan di: /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cmaes_3


### Evaluation

In [89]:
from sacrebleu.metrics import BLEU
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import random
import transformers
import os
from unsloth import FastLanguageModel
from rouge_score import rouge_scorer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    transformers.set_seed(seed)

# System prompt per bahasa
SYSTEM_PROMPT_ID = (
    "Anda adalah asisten AI yang membantu pengguna dalam bahasa Indonesia. "
)

SYSTEM_PROMPT_EN = (
    "You are a helpful AI assistant that communicates in English. "
)

MAX_SEQ_LEN      = 1024   # sama dengan di FastLanguageModel.from_pretrained
MAX_NEW_TOKENS   = 128
MAX_INPUT_LENGTH = MAX_SEQ_LEN - MAX_NEW_TOKENS  # supaya total <= 1024

def make_collate_fn(tokenizer, system_prompt, schema):
    def collate_fn(batch):
        input_id_list = []
        references = []

        for item in batch:
            if schema == "id":
                input_text = item["input"]      # ID: input/output
                reference  = item["output"]
            elif schema == "en":
                input_text = item["question"]   # EN: question/response
                reference  = item["response"]
            else:
                raise ValueError(f"Unknown schema: {schema}")

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": input_text},
            ]

            # 1) Tokenisasi per-sample TANPA padding batch
            ids = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
                truncation=True,
                max_length=MAX_INPUT_LENGTH,
            )[0]  # shape: (seq_len,)

            input_id_list.append(ids)
            references.append(reference)

        # 2) Hitung panjang maksimum di batch
        max_len = max(t.shape[0] for t in input_id_list)
        pad_id  = tokenizer.pad_token_id
        batch_size = len(input_id_list)

        # 3) Siapkan tensor batched dengan LEFT PADDING manual
        input_ids = torch.full(
            (batch_size, max_len),
            pad_id,
            dtype=torch.long,
        )
        attention_mask = torch.zeros(
            (batch_size, max_len),
            dtype=torch.long,
        )

        for i, ids in enumerate(input_id_list):
            seq_len = ids.shape[0]
            # left padding: isi dari kanan
            input_ids[i, -seq_len:] = ids
            attention_mask[i, -seq_len:] = 1

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "references": references,
        }

    return collate_fn

def evaluate_model(model, tokenizer, eval_dataset, system_prompt,
                   batch_size=64, max_allowed_length=24, schema="en"):
    model.eval()
    all_predictions = []
    all_references = []        # untuk BLEU (nested list)
    all_references_text = []   # untuk ROUGE (list string)

    collate_fn = make_collate_fn(tokenizer, system_prompt, schema)

    dataloader = DataLoader(
        eval_dataset,
        num_workers=8,              # 0 dulu biar error kelihatan jelas
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=False,
        pin_memory=True,
    )

    bleu = BLEU()
    pbar = tqdm(dataloader)

    for batch in pbar:
        input_ids = batch["input_ids"].to(DEVICE)          # shape: (B, L)
        attention_mask = batch["attention_mask"].to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_allowed_length,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=False,
                use_cache=True,
            )

        # semua sequence punya panjang awal yang sama: L
        seq_len = input_ids.shape[1]

        predictions = []
        for i, output in enumerate(outputs):
            # ambil token hasil generate saja (setelah input penuh)
            generated_tokens = output[seq_len:]
            decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True)
            predictions.append(decoded)

        all_predictions.extend(predictions)
        all_references.extend([[ref] for ref in batch["references"]])
        all_references_text.extend(batch["references"])

    # ====== BLEU ======
    final_bleu = bleu.corpus_score(all_predictions, all_references)

    # ====== ROUGE-1, ROUGE-2, ROUGE-L ======
    scorer = rouge_scorer.RougeScorer(
        ["rouge1", "rouge2", "rougeL"],
        use_stemmer=True,
    )

    rouge1_list, rouge2_list, rougeL_list = [], [], []

    for pred, ref in zip(all_predictions, all_references_text):
        scores = scorer.score(ref, pred)
        rouge1_list.append(scores["rouge1"].fmeasure)
        rouge2_list.append(scores["rouge2"].fmeasure)
        rougeL_list.append(scores["rougeL"].fmeasure)

    # kali 100 supaya bentuknya persen seperti BLEU
    rouge1 = float(np.mean(rouge1_list) * 100)
    rouge2 = float(np.mean(rouge2_list) * 100)
    rougeL = float(np.mean(rougeL_list) * 100)

    return {
        "bleu": final_bleu.score,
        "rouge1": rouge1,
        "rouge2": rouge2,
        "rougeL": rougeL,
        "predictions": all_predictions,
        "references": all_references,
    }


def evaluate_model_multiple_runs(
    model,
    tokenizer,
    eval_dataset,
    system_prompt,
    n_runs=3,
    batch_size=128,
    max_allowed_length=24,
    schema="en",
):
    all_results = []

    for run in range(n_runs):
        set_seed(42 + run)
        results = evaluate_model(
            model,
            tokenizer,
            eval_dataset,
            system_prompt=system_prompt,
            batch_size=batch_size,
            max_allowed_length=max_allowed_length,
            schema=schema,
        )
        all_results.append(results)

        bleu_scores   = [r["bleu"]   for r in all_results]
        rouge1_scores = [r["rouge1"] for r in all_results]
        rouge2_scores = [r["rouge2"] for r in all_results]
        rougeL_scores = [r["rougeL"] for r in all_results]

        print(
            f"Run {run + 1}/{n_runs} - "
            f"BLEU: {np.mean(bleu_scores):.4f} ± {np.std(bleu_scores):.4f} | "
            f"R1: {np.mean(rouge1_scores):.4f} | "
            f"R2: {np.mean(rouge2_scores):.4f} | "
            f"RL: {np.mean(rougeL_scores):.4f}"
        )

    bleu_scores   = [r["bleu"]   for r in all_results]
    rouge1_scores = [r["rouge1"] for r in all_results]
    rouge2_scores = [r["rouge2"] for r in all_results]
    rougeL_scores = [r["rougeL"] for r in all_results]

    return {
        "bleu_mean":   float(np.mean(bleu_scores)),
        "bleu_std":    float(np.std(bleu_scores)),
        "rouge1_mean": float(np.mean(rouge1_scores)),
        "rouge1_std":  float(np.std(rouge1_scores)),
        "rouge2_mean": float(np.mean(rouge2_scores)),
        "rouge2_std":  float(np.std(rouge2_scores)),
        "rougeL_mean": float(np.mean(rougeL_scores)),
        "rougeL_std":  float(np.std(rougeL_scores)),
    }

MODEL_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cmaes_3"
print(MODEL_DIR)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_DIR,
    max_seq_length=MAX_SEQ_LEN,
    dtype=None,
    load_in_4bit=True,
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.padding_side = "left"

model.eval()
FastLanguageModel.for_inference(model)

results_en_detail = evaluate_model(
    model,
    tokenizer,
    eval_en_clean["test"],
    system_prompt=SYSTEM_PROMPT_EN,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="en",
)

results_id_detail = evaluate_model(
    model,
    tokenizer,
    eval_id_clean["test"],
    system_prompt=SYSTEM_PROMPT_ID,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="id",
)

print("\nFinal Results (test split):")
print(
    f"English (eval_en_clean['test']): "
    f"BLEU = {results_en_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_en_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_en_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_en_detail['rougeL']:.4f}"
)
print(
    f"Indonesian (eval_id_clean['test']): "
    f"BLEU = {results_id_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_id_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_id_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_id_detail['rougeL']:.4f}"
)

/content/drive/MyDrive/College/KK/Final Project/model/lora_soup_cmaes_3
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 34/34 [05:21<00:00,  9.46s/it]



Final Results (test split):
English (eval_en_clean['test']): BLEU = 52.0622, ROUGE-1 = 35.9840, ROUGE-2 = 22.1452, ROUGE-L = 30.1163
Indonesian (eval_id_clean['test']): BLEU = 15.7930, ROUGE-1 = 15.6638, ROUGE-2 = 4.7025, ROUGE-L = 12.6837


## LoRA-Soups (GWO)

### Search for the best weight

In [ ]:
N_HEAD = 10
N_MID = 30
N_TAIL = 10
N = N_HEAD + N_MID + N_TAIL

ds_id = eval_id_clean["test"]
ds_en = eval_en_clean["test"]

n_total = len(ds_id)
assert len(ds_en) == n_total, "Panjang split ID dan EN harus sama"
assert n_total >= N, "Dataset terlalu kecil untuk ambil 50 sampel"

head_indices = list(range(0, N_HEAD))

mid_start = (n_total - N_MID) // 2
mid_indices = list(range(mid_start, mid_start + N_MID))

tail_indices = list(range(n_total - N_TAIL, n_total))

indices = head_indices + mid_indices + tail_indices

test_id_samples = ds_id.select(indices)
test_en_samples = ds_en.select(indices)

display(test_id_samples)
display(test_en_samples)

Dataset({
    features: ['dataset_name', 'subset_name', 'prompt_id', 'template_name', 'dataset_key', 'input', 'output'],
    num_rows: 50
})

Dataset({
    features: ['id', 'system_prompt', 'question', 'response'],
    num_rows: 50
})

In [ ]:
from safetensors.torch import load_file, save_file
import shutil

LORA1_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-id"
LORA2_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-en"

LORA_SOUP_TMP_DIR = "/content/drive/MyDrive/College/KK/Final Project/model/lora_soup_gwo_tmp"

In [ ]:
FLOAT_DTYPES = {
    torch.float16,
    torch.bfloat16,
    torch.float32,
    torch.float64,
}


def load_two_lora_state_dicts(lora_dir_1, lora_dir_2):
    """
    Load state_dict dari 2 adapter LoRA (safetensors).
    Pastikan kedua adapter dilatih dari base dan config yang sama.
    """
    sd1 = load_file(os.path.join(lora_dir_1, "adapter_model.safetensors"))
    sd2 = load_file(os.path.join(lora_dir_2, "adapter_model.safetensors"))

    keys1 = set(sd1.keys())
    keys2 = set(sd2.keys())
    if keys1 != keys2:
        raise ValueError("Key di adapter LoRA 1 dan 2 tidak sama. "
                         "Pastikan keduanya dilatih dari base + config yang sama.")

    return sd1, sd2


def prepare_soup_dir(template_dir, soup_dir):
    """
    Membuat folder soup sekali saja dengan cara copy seluruh isi template_dir
    (misal dari LORA1_DIR) ke soup_dir.
    Nanti setiap kali GA update bobot, kita hanya overwrite adapter_model.safetensors di soup_dir.
    """
    if os.path.exists(soup_dir):
        shutil.rmtree(soup_dir)
    shutil.copytree(template_dir, soup_dir)
    print(f"[INFO] Folder soup dibuat dari template: {template_dir} -> {soup_dir}")


def write_soup_state_dict(sd1, sd2, alpha, out_dir):
    """
    Membuat LoRA-Soup dari 2 adapter:
        W_soup = alpha * W1 + (1 - alpha) * W2

    alpha di [0, 1]. Hasilnya disimpan ke adapter_model.safetensors di out_dir.
    """
    alpha = float(alpha)
    if not (0.0 <= alpha <= 1.0):
        raise ValueError("alpha harus di antara 0 dan 1")

    w1 = alpha
    w2 = 1.0 - alpha

    new_state = {}
    for k in sd1.keys():
        t1 = sd1[k]
        t2 = sd2[k]

        if isinstance(t1, torch.Tensor) and t1.dtype in FLOAT_DTYPES:
            new_state[k] = w1 * t1 + w2 * t2
        else:
            new_state[k] = t1

    out_path = os.path.join(out_dir, "adapter_model.safetensors")
    save_file(new_state, out_path)
    print(f"[DEBUG] Soup weights disimpan ke {out_path}")

In [ ]:
# Sanity Check LoRA
lora_sd1, lora_sd2 = load_two_lora_state_dicts(LORA1_DIR, LORA2_DIR)
prepare_soup_dir(LORA1_DIR, LORA_SOUP_TMP_DIR)

[INFO] Folder soup dibuat dari template: /content/drive/MyDrive/College/KK/Final Project/model/lora-id -> /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_gwo_tmp


In [ ]:
from sacrebleu.metrics import BLEU
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import random
import transformers
import os
from unsloth import FastLanguageModel
from rouge_score import rouge_scorer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    transformers.set_seed(seed)

SYSTEM_PROMPT_ID = (
    "Anda adalah asisten AI yang membantu pengguna dalam bahasa Indonesia. "
)

SYSTEM_PROMPT_EN = (
    "You are a helpful AI assistant that communicates in English. "
)

MAX_SEQ_LEN      = 1024
MAX_NEW_TOKENS   = 128
MAX_INPUT_LENGTH = MAX_SEQ_LEN - MAX_NEW_TOKENS

def make_collate_fn(tokenizer, system_prompt, schema):
    def collate_fn(batch):
        input_id_list = []
        references = []

        for item in batch:
            if schema == "id":
                input_text = item["input"]      # ID: input/output
                reference  = item["output"]
            elif schema == "en":
                input_text = item["question"]   # EN: question/response
                reference  = item["response"]
            else:
                raise ValueError(f"Unknown schema: {schema}")

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": input_text},
            ]

            # 1) Tokenisasi per-sample TANPA padding batch
            ids = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
                truncation=True,
                max_length=MAX_INPUT_LENGTH,
            )[0]  # shape: (seq_len,)

            input_id_list.append(ids)
            references.append(reference)

        # 2) Hitung panjang maksimum di batch
        max_len = max(t.shape[0] for t in input_id_list)
        pad_id  = tokenizer.pad_token_id
        batch_size = len(input_id_list)

        # 3) Siapkan tensor batched dengan LEFT PADDING manual
        input_ids = torch.full(
            (batch_size, max_len),
            pad_id,
            dtype=torch.long,
        )
        attention_mask = torch.zeros(
            (batch_size, max_len),
            dtype=torch.long,
        )

        for i, ids in enumerate(input_id_list):
            seq_len = ids.shape[0]
            # left padding: isi dari kanan
            input_ids[i, -seq_len:] = ids
            attention_mask[i, -seq_len:] = 1

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "references": references,
        }

    return collate_fn

def evaluate_model(model, tokenizer, eval_dataset, system_prompt,
                   batch_size=64, max_allowed_length=24, schema="en"):
    model.eval()
    all_predictions = []
    all_references = []        # untuk BLEU (nested list)
    all_references_text = []   # untuk ROUGE (list string)

    collate_fn = make_collate_fn(tokenizer, system_prompt, schema)

    dataloader = DataLoader(
        eval_dataset,
        num_workers=8,              # 0 dulu biar error kelihatan jelas
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=False,
        pin_memory=True,
    )

    bleu = BLEU()
    pbar = tqdm(dataloader)

    for batch in pbar:
        input_ids = batch["input_ids"].to(DEVICE)          # shape: (B, L)
        attention_mask = batch["attention_mask"].to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_allowed_length,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=False,
                use_cache=True,
            )

        # semua sequence punya panjang awal yang sama: L
        seq_len = input_ids.shape[1]

        predictions = []
        for i, output in enumerate(outputs):
            # ambil token hasil generate saja (setelah input penuh)
            generated_tokens = output[seq_len:]
            decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True)
            predictions.append(decoded)

        all_predictions.extend(predictions)
        all_references.extend([[ref] for ref in batch["references"]])
        all_references_text.extend(batch["references"])

    # ====== BLEU ======
    final_bleu = bleu.corpus_score(all_predictions, all_references)

    # ====== ROUGE-1, ROUGE-2, ROUGE-L ======
    scorer = rouge_scorer.RougeScorer(
        ["rouge1", "rouge2", "rougeL"],
        use_stemmer=True,
    )

    rouge1_list, rouge2_list, rougeL_list = [], [], []

    for pred, ref in zip(all_predictions, all_references_text):
        scores = scorer.score(ref, pred)
        rouge1_list.append(scores["rouge1"].fmeasure)
        rouge2_list.append(scores["rouge2"].fmeasure)
        rougeL_list.append(scores["rougeL"].fmeasure)

    # kali 100 supaya bentuknya persen seperti BLEU
    rouge1 = float(np.mean(rouge1_list) * 100)
    rouge2 = float(np.mean(rouge2_list) * 100)
    rougeL = float(np.mean(rougeL_list) * 100)

    return {
        "bleu": final_bleu.score,
        "rouge1": rouge1,
        "rouge2": rouge2,
        "rougeL": rougeL,
        "predictions": all_predictions,
        "references": all_references,
    }


def evaluate_model_multiple_runs(
    model,
    tokenizer,
    eval_dataset,
    system_prompt,
    n_runs=3,
    batch_size=128,
    max_allowed_length=24,
    schema="en",
):
    all_results = []

    for run in range(n_runs):
        set_seed(42 + run)
        results = evaluate_model(
            model,
            tokenizer,
            eval_dataset,
            system_prompt=system_prompt,
            batch_size=batch_size,
            max_allowed_length=max_allowed_length,
            schema=schema,
        )
        all_results.append(results)

        bleu_scores   = [r["bleu"]   for r in all_results]
        rouge1_scores = [r["rouge1"] for r in all_results]
        rouge2_scores = [r["rouge2"] for r in all_results]
        rougeL_scores = [r["rougeL"] for r in all_results]

        print(
            f"Run {run + 1}/{n_runs} - "
            f"BLEU: {np.mean(bleu_scores):.4f} ± {np.std(bleu_scores):.4f} | "
            f"R1: {np.mean(rouge1_scores):.4f} | "
            f"R2: {np.mean(rouge2_scores):.4f} | "
            f"RL: {np.mean(rougeL_scores):.4f}"
        )

    bleu_scores   = [r["bleu"]   for r in all_results]
    rouge1_scores = [r["rouge1"] for r in all_results]
    rouge2_scores = [r["rouge2"] for r in all_results]
    rougeL_scores = [r["rougeL"] for r in all_results]

    return {
        "bleu_mean":   float(np.mean(bleu_scores)),
        "bleu_std":    float(np.std(bleu_scores)),
        "rouge1_mean": float(np.mean(rouge1_scores)),
        "rouge1_std":  float(np.std(rouge1_scores)),
        "rouge2_mean": float(np.mean(rouge2_scores)),
        "rouge2_std":  float(np.std(rouge2_scores)),
        "rougeL_mean": float(np.mean(rougeL_scores)),
        "rougeL_std":  float(np.std(rougeL_scores)),
    }

def evaluate_alpha(alpha, eval_en_dataset, eval_id_dataset, batch_size=75, verbose=True, weight_id=0.5):
    """
    Evaluasi 1 nilai alpha:
        w1 = alpha, w2 = 1 - alpha
    Fitness = (1 - weight_id)*BLEU_EN + weight_id*BLEU_ID
    """

    # 1. Tulis LoRA-Soup ke LORA_SOUP_TMP_DIR
    write_soup_state_dict(lora_sd1, lora_sd2, alpha, LORA_SOUP_TMP_DIR)

    # 2. Bersihkan GPU sebelum load model baru
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # 3. Load model + tokenizer dari adapter soup
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=LORA_SOUP_TMP_DIR,
        max_seq_length=MAX_SEQ_LEN,
        dtype=None,
        load_in_4bit=True,
    )

    # Set pad token dan padding side sesuai evaluasi sebelumnya
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id
    tokenizer.padding_side = "left"

    model.to(DEVICE)
    model.eval()
    FastLanguageModel.for_inference(model)

    results_en = evaluate_model(
        model,
        tokenizer,
        eval_en_dataset,
        system_prompt=SYSTEM_PROMPT_EN,
        batch_size=batch_size,
        max_allowed_length=MAX_NEW_TOKENS,
        schema="en",
    )

    results_id = evaluate_model(
        model,
        tokenizer,
        eval_id_dataset,
        system_prompt=SYSTEM_PROMPT_ID,
        batch_size=batch_size,
        max_allowed_length=MAX_NEW_TOKENS,
        schema="id",
    )

    bleu_en = results_en["bleu"]
    bleu_id = results_id["bleu"]

    fitness = (1.0 - weight_id) * bleu_en + weight_id * bleu_id

    if verbose:
        print(
            f"[alpha={alpha:.4f}] "
            f"BLEU_EN={bleu_en:.4f}, BLEU_ID={bleu_id:.4f}, "
            f"Fitness={fitness:.4f}"
        )

    del model
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return fitness, bleu_en, bleu_id

In [ ]:
# Quick Test
fit_05, bleu_en_05, bleu_id_05 = evaluate_alpha(
    alpha=0.5,
    eval_en_dataset=test_en_samples,
    eval_id_dataset=test_id_samples,
)
print("Fitness alpha=0.5:", fit_05)

[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_gwo_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.85s/it]


[alpha=0.5000] BLEU_EN=27.9623, BLEU_ID=7.7550, Fitness=17.8587
Fitness alpha=0.5: 17.858654659790623


In [ ]:
import numpy as np

def gwo_optimize_alpha(
    eval_en_dataset,
    eval_id_dataset,
    n_wolves=8,        # jumlah serigala (search agents)
    max_iters=6,      # jumlah iterasi GWO
    a_max=2.0,         # parameter 'a' awal (akan turun ke 0)
    batch_size=80,
    eval_verbose=False,
    seed=42,
    weight_id=0.5,     # bobot pentingnya BLEU ID di evaluate_alpha
):
    """
    Grey Wolf Optimizer (GWO) untuk mencari alpha optimal di [0,1]
    sebagai bobot LoRA-Soup dua adapter.

    Return:
        best_alpha   : alpha terbaik
        best_fitness : fitness terbaik
        history      : list dict berisi log per iterasi (opsional untuk analisis)
    """
    rng = np.random.default_rng(seed)

    # --- 1. Inisialisasi posisi serigala (alpha_i) di [0,1] ---
    positions = rng.uniform(low=0.0, high=1.0, size=(n_wolves,))
    fitness = np.zeros(n_wolves, dtype=float)

    history = []

    # Helper untuk evaluasi dan simpan log
    def eval_position(alpha_value, iter_idx, wolf_idx):
        alpha_value = float(alpha_value)
        fitness_val, bleu_en, bleu_id = evaluate_alpha(
            alpha=alpha_value,
            eval_en_dataset=eval_en_dataset,
            eval_id_dataset=eval_id_dataset,
            batch_size=batch_size,
            verbose=eval_verbose,
            weight_id=weight_id,
        )
        history.append(
            {
                "iter": iter_idx,
                "wolf": wolf_idx,
                "alpha": alpha_value,
                "fitness": float(fitness_val),
                "bleu_en": float(bleu_en),
                "bleu_id": float(bleu_id),
            }
        )
        return fitness_val

    # Fungsi bantu: cari alpha, beta, delta dan fitness-nya
    def get_leaders(positions, fitness):
        # Karena kita MAKSIMALKAN fitness -> sort descending
        idx_sorted = np.argsort(-fitness)
        alpha_pos = positions[idx_sorted[0]]
        beta_pos  = positions[idx_sorted[1]]
        delta_pos = positions[idx_sorted[2]]

        alpha_fit = fitness[idx_sorted[0]]
        beta_fit  = fitness[idx_sorted[1]]
        delta_fit = fitness[idx_sorted[2]]

        return alpha_pos, beta_pos, delta_pos, alpha_fit, beta_fit, delta_fit

    # --- 2. Evaluasi awal semua serigala ---
    for i in range(n_wolves):
        fitness[i] = eval_position(positions[i], iter_idx=0, wolf_idx=i)

    # Tentukan alpha, beta, delta awal
    alpha_pos, beta_pos, delta_pos, alpha_fit, beta_fit, delta_fit = get_leaders(
        positions, fitness
    )

    if eval_verbose:
        print(f"[Iter 00] best_alpha={alpha_pos:.4f}, fitness={alpha_fit:.4f}")

    # --- 4. Loop utama GWO ---
    for iter_idx in range(1, max_iters + 1):
        # Parameter 'a' turun linear dari a_max -> 0
        a = a_max - iter_idx * (a_max / max_iters)

        # Update posisi tiap serigala
        for i in range(n_wolves):
            X = positions[i]  # posisi sekarang (scalar, 1D)

            # Terhadap alpha (leader 1)
            r1, r2 = rng.random(), rng.random()
            A1 = 2 * a * r1 - a
            C1 = 2 * r2
            D_alpha = abs(C1 * alpha_pos - X)
            X1 = alpha_pos - A1 * D_alpha

            # Terhadap beta (leader 2)
            r1, r2 = rng.random(), rng.random()
            A2 = 2 * a * r1 - a
            C2 = 2 * r2
            D_beta = abs(C2 * beta_pos - X)
            X2 = beta_pos - A2 * D_beta

            # Terhadap delta (leader 3)
            r1, r2 = rng.random(), rng.random()
            A3 = 2 * a * r1 - a
            C3 = 2 * r2
            D_delta = abs(C3 * delta_pos - X)
            X3 = delta_pos - A3 * D_delta

            # Posisi baru = rata-rata X1, X2, X3
            new_X = (X1 + X2 + X3) / 3.0

            # Clamp supaya tetap di [0,1]
            new_X = np.clip(new_X, 0.0, 1.0)

            positions[i] = new_X

        # Evaluasi ulang semua posisi setelah update
        for i in range(n_wolves):
            fitness[i] = eval_position(positions[i], iter_idx=iter_idx, wolf_idx=i)

        # Update alpha/beta/delta setelah evaluasi
        alpha_pos, beta_pos, delta_pos, alpha_fit, beta_fit, delta_fit = get_leaders(
            positions, fitness
        )

        if eval_verbose:
            print(
                f"[Iter {iter_idx:02d}] "
                f"best_alpha={alpha_pos:.4f}, fitness={alpha_fit:.4f}"
            )

    # --- 5. Hasil akhir ---
    best_alpha = float(alpha_pos)
    best_fitness = float(alpha_fit)

    return best_alpha, best_fitness, history

In [ ]:
import psutil, os, time

process = psutil.Process(os.getpid())

if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.empty_cache()

start_gpu_mem = torch.cuda.memory_allocated()
start_ram = process.memory_info().rss
start_time = time.perf_counter()

best_alpha, best_fitness, history = gwo_optimize_alpha(
    eval_en_dataset=test_en_samples,
    eval_id_dataset=test_id_samples,
    n_wolves=8,
    max_iters=6,
    batch_size=80,
    eval_verbose=True,
    weight_id=0.5,
)

end_time = time.perf_counter()
end_ram = process.memory_info().rss
end_gpu_mem = torch.cuda.memory_allocated()
peak_gpu_mem = torch.cuda.max_memory_allocated()

elapsed = end_time - start_time

print("\n=== HASIL AKHIR GWO ===")
print(f"alpha terbaik     : {best_alpha:.4f}")
print(f"w_LoRA1 (adapter1): {best_alpha:.4f}")
print(f"w_LoRA2 (adapter2): {1.0 - best_alpha:.4f}")
print(f"fitness (BLEU mix): {best_fitness:.4f}")
print(f"Total waktu GWO   : {elapsed:.2f} detik")

print("\n=== MEMORI GPU ===")
print(f"Memori awal  : {start_gpu_mem / (1024**2):.2f} MB")
print(f"Memori akhir : {end_gpu_mem / (1024**2):.2f} MB")
print(f"Peak usage   : {peak_gpu_mem / (1024**2):.2f} MB")

print("\n=== MEMORI RAM ===")
print(f"RAM awal  : {start_ram / (1024**2):.2f} MB")
print(f"RAM akhir : {end_ram / (1024**2):.2f} MB")
print(f"Δ RAM     : {(end_ram - start_ram) / (1024**2):.2f} MB")

[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_gwo_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.88s/it]


[alpha=0.7740] BLEU_EN=12.2346, BLEU_ID=25.2015, Fitness=18.7180
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_gwo_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.69s/it]


[alpha=0.4389] BLEU_EN=35.3140, BLEU_ID=7.7550, Fitness=21.5345
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_gwo_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

[alpha=0.5632] BLEU_EN=34.6066, BLEU_ID=13.4851, Fitness=24.0459
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_gwo_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.77s/it]


[alpha=0.5632] BLEU_EN=34.6066, BLEU_ID=13.4851, Fitness=24.0459
[DEBUG] Soup weights disimpan ke /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_gwo_tmp/adapter_model.safetensors
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 1/1 [00:21<00:00, 21.83s/it]


[alpha=0.5632] BLEU_EN=34.6066, BLEU_ID=13.4851, Fitness=24.0459
[Iter 06] best_alpha=0.5632, fitness=24.0459

=== HASIL AKHIR GWO ===
alpha terbaik     : 0.5632
w_LoRA1 (adapter1): 0.5632
w_LoRA2 (adapter2): 0.4368
fitness (BLEU mix): 24.0459
Total waktu GWO   : 3420.63 detik

=== MEMORI GPU ===
Memori awal  : 6666.67 MB
Memori akhir : 6641.36 MB
Peak usage   : 12018.73 MB

=== MEMORI RAM ===
RAM awal  : 4876.43 MB
RAM akhir : 4905.10 MB
Δ RAM     : 28.67 MB


### Building the adapter

In [ ]:
# Configuration
BASE_MODEL = "unsloth/Llama-3.2-1B-Instruct-unsloth-bnb-4bit"

LORA1_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-id"
LORA2_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora-en"

LORA_SOUP_DIR = "/content/drive/MyDrive/College/KK/Final Project/model/lora_soup_gwo_1"

CAT_WEIGHTS = [0.5632, 0.4368]

LOAD_IN_4BIT = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

Device: cuda


In [ ]:
def show_adapter_cfg(path):
    with open(os.path.join(path, "adapter_config.json"), "r") as f:
        cfg = json.load(f)
    print("===", path, "===")
    print("r           :", cfg["r"])
    print("lora_alpha  :", cfg.get("lora_alpha"))
    print("target_modules:", cfg.get("target_modules"))

show_adapter_cfg(LORA1_DIR)
show_adapter_cfg(LORA2_DIR)

=== /content/drive/MyDrive/College/KK/Final Project/model/lora-id ===
r           : 16
lora_alpha  : 16
target_modules: ['k_proj', 'o_proj', 'up_proj', 'down_proj', 'v_proj', 'gate_proj', 'q_proj']
=== /content/drive/MyDrive/College/KK/Final Project/model/lora-en ===
r           : 16
lora_alpha  : 16
target_modules: ['q_proj', 'o_proj', 'down_proj', 'v_proj', 'up_proj', 'k_proj', 'gate_proj']


In [ ]:
import os, json, shutil
from typing import List, Optional

import torch
from safetensors.torch import load_file, save_file

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def build_lora_soup(
    lora_paths: List[str],
    weights: Optional[List[float]],
    out_path: str,
    copy_tokenizer: bool = True,
):
    """
    Menggabungkan beberapa LoRA menjadi 1 adapter LoRA-Soups
    dengan cara *weighted average* (model soup) atas seluruh
    bobot float di state_dict.

    lora_paths : list path adapter LoRA sumber
    weights    : bobot soup, akan dinormalisasi; jika None -> uniform
    out_path   : folder output adapter baru
    """
    assert len(lora_paths) >= 2, "Minimal 2 LoRA untuk LoRA-Soups"

    # jika weights tidak diberikan -> uniform
    if weights is None:
        w = torch.ones(len(lora_paths), dtype=torch.float32)
    else:
        assert len(weights) == len(lora_paths), "Panjang weights harus = jumlah LoRA"
        w = torch.tensor(weights, dtype=torch.float32)

    # normalisasi bobot
    w = w / w.sum()

    os.makedirs(out_path, exist_ok=True)

    # 1. baca config dari adapter pertama
    cfg_path0 = os.path.join(lora_paths[0], "adapter_config.json")
    with open(cfg_path0, "r") as f:
        cfg0 = json.load(f)

    base_r      = cfg0["r"]
    base_alpha  = cfg0.get("lora_alpha", None)
    base_target = set(cfg0.get("target_modules", []))

    # cek konsistensi adapter lain
    for p in lora_paths[1:]:
        with open(os.path.join(p, "adapter_config.json"), "r") as f:
            cfgi = json.load(f)
        assert cfgi["r"] == base_r, f"Rank LoRA beda di {p}"
        if "lora_alpha" in cfgi and base_alpha is not None:
            assert cfgi["lora_alpha"] == base_alpha, f"lora_alpha beda di {p}"
        if "target_modules" in cfgi and base_target:
            assert set(cfgi["target_modules"]) == base_target, f"target_modules beda di {p}"

    # 2. config baru untuk SOUP:
    #    rank r & lora_alpha TIDAK berubah, hanya bobot yang di-average
    cfg_soup = cfg0.copy()
    if "target_modules" in cfg_soup:
        cfg_soup["target_modules"] = sorted(cfg_soup["target_modules"])

    with open(os.path.join(out_path, "adapter_config.json"), "w") as f:
        json.dump(cfg_soup, f, indent=2)

    # 3. load semua state dict LoRA
    state_dicts = [load_file(os.path.join(p, "adapter_model.safetensors")) for p in lora_paths]

    # opsional: pastikan key sama semua
    keys0 = set(state_dicts[0].keys())
    for i, sd in enumerate(state_dicts[1:], start=1):
        ki = set(sd.keys())
        assert keys0 == ki, f"Kumpulan key state_dict LoRA ke-{i} beda; cek adapter: {lora_paths[i]}"

    # 4. build state dict baru dengan weighted average
    new_state = {}
    float_dtypes = {
        torch.float16,
        torch.bfloat16,
        torch.float32,
        torch.float64,
    }

    for key in state_dicts[0].keys():
        tensor0 = state_dicts[0][key]

        # Kalau tensor floating -> soup (weighted average)
        if isinstance(tensor0, torch.Tensor) and tensor0.dtype in float_dtypes:
            acc = torch.zeros_like(tensor0)
            for wi, sd in zip(w, state_dicts):
                acc = acc + wi * sd[key]
            new_state[key] = acc
        else:
            # Non-float (misal buffer int, dsb) -> ambil dari adapter pertama saja
            new_state[key] = tensor0

    # 5. simpan
    save_file(new_state, os.path.join(out_path, "adapter_model.safetensors"))

    # 6. copy tokenizer & template dari adapter pertama (opsional)
    if copy_tokenizer:
        extra_files = [
            "tokenizer.json",
            "tokenizer_config.json",
            "special_tokens_map.json",
            "chat_template.jinja",
            "README.md",
        ]
        for fname in extra_files:
            src = os.path.join(lora_paths[0], fname)
            if os.path.exists(src):
                shutil.copy(src, os.path.join(out_path, fname))

    print(f"[OK] LoRA-Soups disimpan di: {out_path}")

In [ ]:
build_lora_soup(
    lora_paths=[LORA1_DIR, LORA2_DIR],
    weights=CAT_WEIGHTS,
    out_path=LORA_SOUP_DIR,
    copy_tokenizer=True,
)

[OK] LoRA-Soups disimpan di: /content/drive/MyDrive/College/KK/Final Project/model/lora_soup_gwo_1


### Evaluation

In [87]:
from sacrebleu.metrics import BLEU
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import random
import transformers
import os
from unsloth import FastLanguageModel
from rouge_score import rouge_scorer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    transformers.set_seed(seed)

# System prompt per bahasa
SYSTEM_PROMPT_ID = (
    "Anda adalah asisten AI yang membantu pengguna dalam bahasa Indonesia. "
)

SYSTEM_PROMPT_EN = (
    "You are a helpful AI assistant that communicates in English. "
)

MAX_SEQ_LEN      = 1024   # sama dengan di FastLanguageModel.from_pretrained
MAX_NEW_TOKENS   = 128
MAX_INPUT_LENGTH = MAX_SEQ_LEN - MAX_NEW_TOKENS  # supaya total <= 1024

def make_collate_fn(tokenizer, system_prompt, schema):
    def collate_fn(batch):
        input_id_list = []
        references = []

        for item in batch:
            if schema == "id":
                input_text = item["input"]      # ID: input/output
                reference  = item["output"]
            elif schema == "en":
                input_text = item["question"]   # EN: question/response
                reference  = item["response"]
            else:
                raise ValueError(f"Unknown schema: {schema}")

            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": input_text},
            ]

            # 1) Tokenisasi per-sample TANPA padding batch
            ids = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
                truncation=True,
                max_length=MAX_INPUT_LENGTH,
            )[0]  # shape: (seq_len,)

            input_id_list.append(ids)
            references.append(reference)

        # 2) Hitung panjang maksimum di batch
        max_len = max(t.shape[0] for t in input_id_list)
        pad_id  = tokenizer.pad_token_id
        batch_size = len(input_id_list)

        # 3) Siapkan tensor batched dengan LEFT PADDING manual
        input_ids = torch.full(
            (batch_size, max_len),
            pad_id,
            dtype=torch.long,
        )
        attention_mask = torch.zeros(
            (batch_size, max_len),
            dtype=torch.long,
        )

        for i, ids in enumerate(input_id_list):
            seq_len = ids.shape[0]
            # left padding: isi dari kanan
            input_ids[i, -seq_len:] = ids
            attention_mask[i, -seq_len:] = 1

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "references": references,
        }

    return collate_fn

def evaluate_model(model, tokenizer, eval_dataset, system_prompt,
                   batch_size=64, max_allowed_length=24, schema="en"):
    model.eval()
    all_predictions = []
    all_references = []        # untuk BLEU (nested list)
    all_references_text = []   # untuk ROUGE (list string)

    collate_fn = make_collate_fn(tokenizer, system_prompt, schema)

    dataloader = DataLoader(
        eval_dataset,
        num_workers=8,              # 0 dulu biar error kelihatan jelas
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=False,
        pin_memory=True,
    )

    bleu = BLEU()
    pbar = tqdm(dataloader)

    for batch in pbar:
        input_ids = batch["input_ids"].to(DEVICE)          # shape: (B, L)
        attention_mask = batch["attention_mask"].to(DEVICE)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=max_allowed_length,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=False,
                use_cache=True,
            )

        # semua sequence punya panjang awal yang sama: L
        seq_len = input_ids.shape[1]

        predictions = []
        for i, output in enumerate(outputs):
            # ambil token hasil generate saja (setelah input penuh)
            generated_tokens = output[seq_len:]
            decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True)
            predictions.append(decoded)

        all_predictions.extend(predictions)
        all_references.extend([[ref] for ref in batch["references"]])
        all_references_text.extend(batch["references"])

    # ====== BLEU ======
    final_bleu = bleu.corpus_score(all_predictions, all_references)

    # ====== ROUGE-1, ROUGE-2, ROUGE-L ======
    scorer = rouge_scorer.RougeScorer(
        ["rouge1", "rouge2", "rougeL"],
        use_stemmer=True,
    )

    rouge1_list, rouge2_list, rougeL_list = [], [], []

    for pred, ref in zip(all_predictions, all_references_text):
        scores = scorer.score(ref, pred)
        rouge1_list.append(scores["rouge1"].fmeasure)
        rouge2_list.append(scores["rouge2"].fmeasure)
        rougeL_list.append(scores["rougeL"].fmeasure)

    # kali 100 supaya bentuknya persen seperti BLEU
    rouge1 = float(np.mean(rouge1_list) * 100)
    rouge2 = float(np.mean(rouge2_list) * 100)
    rougeL = float(np.mean(rougeL_list) * 100)

    return {
        "bleu": final_bleu.score,
        "rouge1": rouge1,
        "rouge2": rouge2,
        "rougeL": rougeL,
        "predictions": all_predictions,
        "references": all_references,
    }


def evaluate_model_multiple_runs(
    model,
    tokenizer,
    eval_dataset,
    system_prompt,
    n_runs=3,
    batch_size=128,
    max_allowed_length=24,
    schema="en",
):
    all_results = []

    for run in range(n_runs):
        set_seed(42 + run)
        results = evaluate_model(
            model,
            tokenizer,
            eval_dataset,
            system_prompt=system_prompt,
            batch_size=batch_size,
            max_allowed_length=max_allowed_length,
            schema=schema,
        )
        all_results.append(results)

        bleu_scores   = [r["bleu"]   for r in all_results]
        rouge1_scores = [r["rouge1"] for r in all_results]
        rouge2_scores = [r["rouge2"] for r in all_results]
        rougeL_scores = [r["rougeL"] for r in all_results]

        print(
            f"Run {run + 1}/{n_runs} - "
            f"BLEU: {np.mean(bleu_scores):.4f} ± {np.std(bleu_scores):.4f} | "
            f"R1: {np.mean(rouge1_scores):.4f} | "
            f"R2: {np.mean(rouge2_scores):.4f} | "
            f"RL: {np.mean(rougeL_scores):.4f}"
        )

    bleu_scores   = [r["bleu"]   for r in all_results]
    rouge1_scores = [r["rouge1"] for r in all_results]
    rouge2_scores = [r["rouge2"] for r in all_results]
    rougeL_scores = [r["rougeL"] for r in all_results]

    return {
        "bleu_mean":   float(np.mean(bleu_scores)),
        "bleu_std":    float(np.std(bleu_scores)),
        "rouge1_mean": float(np.mean(rouge1_scores)),
        "rouge1_std":  float(np.std(rouge1_scores)),
        "rouge2_mean": float(np.mean(rouge2_scores)),
        "rouge2_std":  float(np.std(rouge2_scores)),
        "rougeL_mean": float(np.mean(rougeL_scores)),
        "rougeL_std":  float(np.std(rougeL_scores)),
    }

MODEL_DIR  = "/content/drive/MyDrive/College/KK/Final Project/model/lora_soup_gwo_1"
print(MODEL_DIR)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_DIR,
    max_seq_length=MAX_SEQ_LEN,
    dtype=None,
    load_in_4bit=True,
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

tokenizer.padding_side = "left"

model.eval()
FastLanguageModel.for_inference(model)

results_en_detail = evaluate_model(
    model,
    tokenizer,
    eval_en_clean["test"],
    system_prompt=SYSTEM_PROMPT_EN,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="en",
)

results_id_detail = evaluate_model(
    model,
    tokenizer,
    eval_id_clean["test"],
    system_prompt=SYSTEM_PROMPT_ID,
    batch_size=90,
    max_allowed_length=MAX_NEW_TOKENS,
    schema="id",
)

print("\nFinal Results (test split):")
print(
    f"English (eval_en_clean['test']): "
    f"BLEU = {results_en_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_en_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_en_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_en_detail['rougeL']:.4f}"
)
print(
    f"Indonesian (eval_id_clean['test']): "
    f"BLEU = {results_id_detail['bleu']:.4f}, "
    f"ROUGE-1 = {results_id_detail['rouge1']:.4f}, "
    f"ROUGE-2 = {results_id_detail['rouge2']:.4f}, "
    f"ROUGE-L = {results_id_detail['rougeL']:.4f}"
)

/content/drive/MyDrive/College/KK/Final Project/model/lora_soup_gwo_1
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


100%|██████████| 34/34 [05:22<00:00,  9.47s/it]



Final Results (test split):
English (eval_en_clean['test']): BLEU = 54.8866, ROUGE-1 = 34.0095, ROUGE-2 = 20.5274, ROUGE-L = 28.6973
Indonesian (eval_id_clean['test']): BLEU = 19.0596, ROUGE-1 = 16.6675, ROUGE-2 = 5.4307, ROUGE-L = 13.9907
